# AInewsbot.ipynb - Automate collecting daily AI news

1. initial scrape of front pages of tech sites
  - Open URLs of news sites specififed in `sources` dict (sources.yaml) using Selenium and Firefox
  - Save HTML of each URL in htmldata directory  
  - Extract URLs from all files, create a pandas dataframe with url, title, src

2. Filter and clean to AI-related headlines not seen before
  - Use ChatGPT prompt to filter only AI-related headlines by sending a prompt and formatted table of headlines
  - Use SQLite to filter headlines previously seen 
  - remove duplicate URLs and headlines
  - ensure there are pretty source names for each news site

3. Topic analysis, make a list of topics for each headline
  - using a prompt, check each headline against a number of evergreen AI topics, e.g. deepfakes, regulation, AI in education
  - extract free from topics from each headline
  - combine topics into topic list for each headline
  - cluster headlines using dimensionality-reduced embeddings and DBSCAN; ask chatgpt to name each cluster
  - sort headlines by doing a traveling salesman shortest traversal in embedding space

4. Summarize individual news story pages in 3 bullets using a prompt

5. create a large markdown file with all bullet points and topics

6. give the markdown file to ChatGPT and ask it to make a list of most popular and import topics of the day

7. human should make a list of the day's topics, combining the chatgpt response to the quesion and cluster topics and 

8. Put summaries in vector store along with metadata. For each topic, retrieve all associated stories and have chatgpt write a digest of those stories in the given format.

9. assemble stories into first draft of newsletter for rewriting as necessary

todo:

use langgraph for final editing workfow
1. prompt to edit final copy for dupes, combine similar sections, copy edit
2. have a reviewer prompt check if there are any bullet points to move to a different section 
3. if so have an editor prompt remove them , return to 2. until nothing left to move dupes left
4. have a reviewer check each section, identify bullet points that are similar to other bullet points in the section and have identical links. rewrite combining so there is no duplication. 
5. identify sections that are short or similar to other sections and suggest sections that should be combined them
6. have an editor prompt merge short sections, return to 4, until no orphan sections left
7. maybe final copy-edit prompt

Original, alternative manual workflow to get HTML files if necessary
- Use Chrome, open e.g. Tech News bookmark folder, right-click and open all bookmarks in new window
- on Google News, make sure switch to AI tab
- on Google News, Feedly, Reddit, scroll to additional pages as desired
- Use SingleFile extension, 'save all tabs'
- Move files to htmldata directory
- Run lower part of notebook to process the data


In [126]:
# import sys
# del sys.modules['ainb_utilities']


In [127]:
from datetime import datetime
import os
import yaml
import dotenv
import sqlite3
import unicodedata
import json
import pickle
from collections import Counter
import shutil

import numpy as np
import pandas as pd
import umap
# import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

# import bs4
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import trafilatura

import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed
import asyncio
import aiohttp

from IPython.display import HTML, Image, Markdown, display
import markdown

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from langchain.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings

from openai import OpenAI

from ainb_const import (DOWNLOAD_DIR, LOWCOST_MODEL, MODEL, CANONICAL_TOPICS,
                        SOURCECONFIG, FILTER_PROMPT, TOPIC_PROMPT,
                        SUMMARIZE_SYSTEM_PROMPT, SUMMARIZE_USER_PROMPT, FINAL_SUMMARY_PROMPT, TOP_CATEGORIES_PROMPT,
                        MAX_INPUT_TOKENS, MAX_OUTPUT_TOKENS, MAX_RETRIES, TEMPERATURE)
from ainb_utilities import (log, delete_files, filter_unseen_urls_db, insert_article, 
                            nearest_neighbor_sort, agglomerative_cluster_sort, traveling_salesman_sort_scipy,
                            unicode_to_ascii, send_gmail)
from ainb_webscrape import (get_driver, quit_drivers, launch_drivers, get_file, get_url, parse_file, 
                            get_og_tags, get_path_from_url, trimmed_href, process_source_queue_factory, 
                            process_url_queue_factory, DRIVERS)
from ainb_llm import (paginate_df, process_pages, fetch_pages, fetch_openai, fetch_all_summaries, 
                      fetch_openai_summary, trunc_tokens, categorize_headline)


import asyncio
# need this to run async in jupyter since it already has an asyncio event loop running
import nest_asyncio
nest_asyncio.apply()


# Initialize

In [128]:
# OpenAI API module
client = OpenAI()

# Or can use REST API directly
API_URL = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {os.getenv("OPENAI_API_KEY")}',
}


In [129]:
#  load sources to scrape from sources.yaml
with open(SOURCECONFIG, "r") as stream:
    try:
        sources = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

log(f"Load {len(sources)} sources from {SOURCECONFIG}")

# make a reverse dict to map output file titles to source names
sources_reverse = {}
for k, v in sources.items():
    log(f"{k} -> {v['url']} -> {v['title']}.html")
    v['sourcename'] = k
    # map filename (title) to source name
    sources_reverse[v['title']] = k

log(f"Mapped {len(sources_reverse)} source page titles to sources")


2024-07-16 00:03:27,462 - AInewsbot - INFO - Load 17 sources from sources.yaml
2024-07-16 00:03:27,463 - AInewsbot - INFO - Ars Technica -> https://arstechnica.com/ -> Ars Technica.html
2024-07-16 00:03:27,463 - AInewsbot - INFO - Bloomberg Tech -> https://www.bloomberg.com/technology -> Bloomberg Technology - Bloomberg.html
2024-07-16 00:03:27,464 - AInewsbot - INFO - Business Insider -> https://www.businessinsider.com/tech -> Tech - Business Insider.html
2024-07-16 00:03:27,464 - AInewsbot - INFO - FT Tech -> https://www.ft.com/technology -> Technology.html
2024-07-16 00:03:27,464 - AInewsbot - INFO - Feedly AI -> https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC9jYXRlZ29yeS9HYWRnZXRzIn1dfQ -> Discover and Add New Feedly AI Feeds.html
2024-07-16 00:0

20

In [130]:
sources

{'Ars Technica': {'include': ['^https://arstechnica.com/(\\w+)/(\\d+)/(\\d+)/'],
  'title': 'Ars Technica',
  'url': 'https://arstechnica.com/',
  'sourcename': 'Ars Technica'},
 'Bloomberg Tech': {'include': ['^https://www.bloomberg.com/news/'],
  'title': 'Bloomberg Technology - Bloomberg',
  'url': 'https://www.bloomberg.com/technology',
  'sourcename': 'Bloomberg Tech'},
 'Business Insider': {'exclude': ['^https://www.insider.com',
   '^https://www.passionfroot.me'],
  'title': 'Tech - Business Insider',
  'url': 'https://www.businessinsider.com/tech',
  'sourcename': 'Business Insider'},
 'FT Tech': {'include': ['https://www.ft.com/content/'],
  'title': 'Technology',
  'url': 'https://www.ft.com/technology',
  'sourcename': 'FT Tech'},
 'Feedly AI': {'exclude': ['^https://feedly.com',
   '^https://s1.feedly.com',
   '^https://blog.feedly.com'],
  'scroll': 5,
  'initial_sleep': 30,
  'title': 'Discover and Add New Feedly AI Feeds',
  'url': 'https://feedly.com/i/aiFeeds?options=e

In [131]:
sources_reverse


{'Ars Technica': 'Ars Technica',
 'Bloomberg Technology - Bloomberg': 'Bloomberg Tech',
 'Tech - Business Insider': 'Business Insider',
 'Technology': 'FT Tech',
 'Discover and Add New Feedly AI Feeds': 'Feedly AI',
 'Google News - Technology - Artificial intelligence': 'Google News',
 'Hacker News Page 1': 'Hacker News',
 'Hacker News Page 2': 'Hacker News 2',
 'HackerNoon - read, write and learn about any technology': 'HackerNoon',
 'Technology - The New York Times': 'NYT Tech',
 'top scoring links _ multi': 'Reddit',
 'Techmeme': 'Techmeme',
 'The Register_ Enterprise Technology News and Analysis': 'The Register',
 'Artificial Intelligence - The Verge': 'The Verge',
 'AI News _ VentureBeat': 'VentureBeat',
 'Technology - WSJ.com': 'WSJ Tech',
 'Technology - The Washington Post': 'WaPo Tech'}

In [132]:
# determine files already in htmldata directory
# List all paths in the directory matching today's date
nfiles = 50
files = [os.path.join(DOWNLOAD_DIR, file)
         for file in os.listdir(DOWNLOAD_DIR)]
# Get the current date
today = datetime.now()
year, month, day = today.year, today.month, today.day
datestr = datetime.now().strftime("%m_%d_%Y")

# filter files only
files = [file for file in files if os.path.isfile(file)]

# Sort files by modification time and take top 50
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
file = files[:nfiles]

# filter files by with today's date ending in .html
files = [
    file for file in files if datestr in file and file.endswith(".html")]
log(len(files))
for file in files:
    log(file)

saved_pages = []
for file in files:
    filename = os.path.basename(file)
    # locate date like '01_14_2024' in filename
    position = filename.find(" (" + datestr)
    basename = filename[:position]
    # match to source name
    sourcename = sources_reverse.get(basename)
    if sourcename is None:
        log(f"Skipping {basename}, no sourcename metadata")
        continue
    sources[sourcename]['latest'] = file
    saved_pages.append((sourcename, file))

2024-07-16 00:03:27,489 - AInewsbot - INFO - 0


# Fetch and save source pages

In [133]:
# Fetch HTML files from sources

# empty download directory
delete_files(DOWNLOAD_DIR)

# save each file specified from sources
num_browsers = 3
log(f"Saving HTML files using {num_browsers} browsers")

# Create a queue for multiprocessing and populate it 
queue = multiprocessing.Queue()
for item in sources.values():
    queue.put(item)
    
# Function to take the queue and pop entries off and process until none are left
# lets you create an array of functions with different args
callable = process_source_queue_factory(queue)

saved_pages = launch_drivers(num_browsers, callable)


2024-07-16 00:03:27,537 - AInewsbot - INFO - Saving HTML files using 3 browsers
2024-07-16 00:03:27,551 - AInewsbot - INFO - get_driver - 17994 Initializing webdriver
2024-07-16 00:03:27,561 - AInewsbot - INFO - get_driver - 17994 Initializing webdriver
2024-07-16 00:03:27,561 - AInewsbot - INFO - get_driver - 17994 Initializing webdriver
2024-07-16 00:03:46,934 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-16 00:03:46,937 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-16 00:03:47,362 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-16 00:03:47,363 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-16 00:03:47,645 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-16 00:03:47,645 - AInewsbot - INFO - get_driver - Initialized webdriver service

KeyboardInterrupt

2024-07-16 00:04:23,300 - urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, 

In [ ]:
log(f"Saved {len(saved_pages)} pages")

print(len(saved_pages))
for sourcename, page in saved_pages:
    sources[sourcename]['latest'] = page
    log(f"{sourcename} -> {page}")
    

# Extract news URLs from saved pages

In [ ]:
# Parse news URLs and titles from downloaded HTML files
log("Parsing html files")
all_urls = []
for sourcename, filename in saved_pages:
    log(sourcename +' -> ' + filename)
    log(f"{sourcename}", "parse loop")
    links = parse_file(sources[sourcename])
    log(f"{len(links)} links found", "parse loop")
    all_urls.extend(links)

log(f"found {len(all_urls)} links", "parse loop")

# make a pandas dataframe of all the links found
orig_df = (
    pd.DataFrame(all_urls)
    .groupby("url")
    .first()
    .reset_index()
    .sort_values("src")[["src", "title", "url"]]
    .reset_index(drop=True)
    .reset_index(drop=False)
    .rename(columns={"index": "id"})
)
orig_df.head()


In [ ]:
# map google news headlines to redirect
def get_google_news_redirects(orig_df):
    redirect_dict = {}
    for row in orig_df.itertuples():
        parsed_url = urlparse(row.url)
        netloc = parsed_url.netloc
        if netloc == 'news.google.com':
            logstr = netloc + " -> "
            response = requests.get(row.url, allow_redirects=False)
            # The URL to which it would have redirected
            redirect_url = response.headers.get('Location')
            redirect_dict[row.url] = redirect_url
            parsed_url2 = urlparse(redirect_url)
            netloc2 = parsed_url2.netloc
            if netloc2 == 'news.google.com':
#                 logstr += netloc2 + " -> "
                response = requests.get(redirect_url, allow_redirects=False)
            # The URL to which it would have redirected
                redirect_url = response.headers.get('Location')
                redirect_dict[row.url] = redirect_url
                logstr += redirect_url
            log(logstr, "get_google_news_redirects")
    return redirect_dict

redirect_dict = get_google_news_redirects(orig_df)

orig_df['actual_url'] = orig_df['url'].apply(lambda url: redirect_dict.get(url, url))


In [ ]:
# # extracts all links from history where isAI=1
# # useful for training dimensionality reduction
# conn = sqlite3.connect('articles.db')
# c = conn.cursor()
# #  and timestamp > '2024-07-01' 
# query = "select * from news_articles where isAI=1 order by id"
# ai_history_df = pd.read_sql_query(query, conn)
# ai_history_df

# Filter URLs to new AI headlines only

In [ ]:
# filter urls we've already seen in previous runs and saved in SQLite
filtered_df = filter_unseen_urls_db(orig_df, before_date='2024-07-15 14:00:00')
len(filtered_df)

In [ ]:
# use chatgpt to filter AI-related headlines using a prompt to OpenAI
print(FILTER_PROMPT)


In [ ]:
# make pages that fit in a reasonably sized (MAXPAGELEN or MAX_INPUT_TOKENS) prompt
pages = paginate_df(filtered_df)
log(f"Paginated {len(pages)} pages")


In [ ]:
# use REST API directly. OpenAI python API doesn't support concurrent requests from a single client
# this runs fast with async aiohttp and on gpt-3.5 (15 seconds vs 2 minutes synchronously with gpt-4o)
# the old API supported submitting multiple payloads in a single completion request
# current API supports a slow 'batch' submission https://platform.openai.com/docs/guides/rate-limits/usage-tiers
# there is a more complex example here - https://github.com/openai/openai-cookbook/blob/main/examples/api_request_parallel_processor.py

log("start classify")
enriched_urls = asyncio.run(fetch_pages(pages, prompt=FILTER_PROMPT))
log("end classify")

enriched_df = pd.DataFrame(enriched_urls)
print(len(enriched_df))
log("isAI", len(enriched_df.loc[enriched_df["isAI"]]))
log("not isAI", len(enriched_df.loc[~enriched_df["isAI"]]))
enriched_df.head()


In [ ]:
# merge returned df with isAI column into original df on id column
merged_df = pd.merge(filtered_df, enriched_df, on="id", how="outer")
merged_df['date'] = datetime.now().date()
merged_df.head()


In [ ]:
# should be empty, shouldn't get back rows that don't match to existing
log(f"Unmatched response rows: {len(merged_df.loc[merged_df['src'].isna()])}")
# should be empty, should get back all rows from orig
log(f"Unmatched source rows: {len(merged_df.loc[merged_df['isAI'].isna()])}")


In [ ]:
# must do this after fixing google actualurl
merged_df['hostname']=merged_df['actual_url'].apply(lambda url: urlparse(url).netloc)
merged_df.head()

In [ ]:
# update SQLite database with all seen URLs
conn = sqlite3.connect('articles.db')
cursor = conn.cursor()

for row in merged_df.itertuples():
    insert_article(conn, cursor, row.src, row.hostname, row.title,
                   row.url, row.actual_url, row.isAI, row.date)


In [ ]:
# keep headlines that are related to AI
AIdf = merged_df.loc[merged_df["isAI"]==1] \
    .reset_index(drop=True)  \
    .reset_index()  \
    .drop(columns=["id"])  \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} AI headlines")
AIdf

In [ ]:
# map title to ascii characters to avoid some dupes with e.g. different quote symbols

def unicode_to_ascii(input_string):
    # Normalize the Unicode string to NFKD form
    normalized_string = unicodedata.normalize('NFKD', input_string)
    
    # Encode to ASCII bytes, ignoring characters that cannot be converted
    ascii_bytes = normalized_string.encode('ascii', 'ignore')
    
    # Convert bytes back to a string
    ascii_string = ascii_bytes.decode('ascii')
    
    return ascii_string

AIdf['title'] = AIdf['title'].apply(unicode_to_ascii)


In [ ]:
# dedupe identical URLs, Google News and Feedly might have same URLs with different headlines
AIdf = AIdf.sort_values("src") \
    .groupby("url") \
    .first() \
    .reset_index(drop=False) \
    .drop(columns=['id']) \
    .reset_index() \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} unique AI URLs")


In [ ]:
# dedupe identical headlines
AIdf['title_clean'] = AIdf['title'].map(lambda s: "".join(s.split()))
AIdf = AIdf.sort_values("src") \
    .groupby("title_clean") \
    .first() \
    .reset_index(drop=True) \
    .drop(columns=['id']) \
    .reset_index() \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} unique AI headlines")


### Get site names and update site names based on URL

In [ ]:
# get site_name
conn = sqlite3.connect('articles.db')
c = conn.cursor()
#  and timestamp > '2024-07-01' 
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.hostname:row.site_name for row in sites_df.itertuples()}

sites_df

In [ ]:
AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, ""))
AIdf.loc[AIdf['site_name']==""]

In [ ]:
async def get_site_name(session, row):
    cat_prompt = f"""
based on this url and your knowledge of the Web, what is the name of the site? https://{row.hostname}

return the response as a json object of the form {{"url": "www.yankodesign.com", "site_name": "Yanko Design"}}

    """
    try:
        messages=[
                  {"role": "user", "content": cat_prompt
                  }]

        payload = {"model":  LOWCOST_MODEL,
                   "response_format": {"type": "json_object"},
                   "messages": messages,
                   "temperature": 0
                   }
        response = await fetch_openai(session, payload)
        response_dict = json.loads(response["choices"][0]["message"]["content"])
        return response_dict
    except Exception as exc:
        print(exc)
                
tasks = []
async with aiohttp.ClientSession() as session:
    for row in AIdf.loc[AIdf['site_name']==""].itertuples():
        task = asyncio.create_task(get_site_name(session, row))
        tasks.append(task)
    responses = await asyncio.gather(*tasks)

responses


In [ ]:
# update site_dict from responses
new_urls = []
for r in responses:
    if r['url'].startswith('https://'):
        r['url'] = r['url'][8:]
    new_urls.append(r['url'])
    sites_dict[r['url']] = r['site_name']
    print(r['url'], r['site_name'])

AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, hostname))



In [ ]:
for url in new_urls:
    sqlstr = "INSERT OR IGNORE INTO sites (hostname, site_name) VALUES (?, ?);"
    print(url, '->', sites_dict[url])
    conn.execute(sqlstr, (url, sites_dict[url]))
    conn.commit()


# Topic analysis
Try to identify the top topics of the day, to help make a nice summary. 

1st approach - do dimensionality reduction on the headline embeddings with UMAP and cluster with DBSCAN.

2nd approach
 - extract topics from headline using a prompt
 - human canonicalizes topics
 - assign headlines to topics using a prompt
 
 The final summary is pretty inconsistent, would be nice to give chatgpt a prompt that would say, summarize these bullet points using this categorization.
 

### Fit dimensionality reduction model

In [ ]:
# # train dimensionality reduction, only need to do this every few months and pickle the model to reflect new topics
# # extracts all links from history where isAI=1
# conn = sqlite3.connect('articles.db')
# c = conn.cursor()
# #  and timestamp > '2024-07-01' 
# query = "select * from news_articles where isAI=1 order by id desc limit 20000"
# ai_history_df = pd.read_sql_query(query, conn)
# len(ai_history_df)

In [ ]:
# embedding_model="text-embedding-3-large"
# embedding_df_list = []
# pages = paginate_df(ai_history_df, maxpagelen=1000, max_input_tokens=8192)

# for p in pages:
#     response = client.embeddings.create(input=[obj['title'] for obj in p],
#                                         model=embedding_model)
#     embedding_df_list.append(pd.DataFrame([e.model_dump()['embedding'] for e in response.data]))

# embedding_df = pd.concat(embedding_df_list, axis=0, ignore_index=True)

# embedding_df.to_pickle("historical_embedding_df.pkl")


In [ ]:
# # Initialize the UMAP reducer
# reducer = umap.UMAP(n_components=30)
# # Fit the reducer to the data without transforming
# reducer.fit(embedding_df)
# # Pickle the reducer
# with open('reducer.pkl', 'wb') as f:
#     pickle.dump(reducer, f)
# print("UMAP reducer pickled and saved as 'reducer.pkl'")

In [ ]:
# attempt to extract top topics 
print(TOPIC_PROMPT)


In [ ]:
# get topics
pages = paginate_df(AIdf)

# apply this prompt to AI headlines
log("start topic extraction")
response = asyncio.run(fetch_pages(pages, prompt=TOPIC_PROMPT))
log("end topic extraction")

In [ ]:
topic_df = pd.DataFrame(response)
topic_df = topic_df.rename(columns={'topics': 'extracted_topics'})
print(len(topic_df))
topic_df.head()


In [ ]:
all_topics = [item.lower() for row in topic_df.itertuples() for item in row.extracted_topics]
item_counts = Counter(all_topics)
filtered_topics = [item for item in item_counts if item_counts[item] >= 2 and item not in {'technology', 'ai', 'artificial intelligence'}]
print(len(filtered_topics))
sorted(filtered_topics)


In [ ]:
topic_df['extracted_topics'] = topic_df['extracted_topics'].apply(lambda l: [t.title() for t in l if t.lower() in filtered_topics])

In [ ]:
# evergreen topics to hopefully map healdines to canonical standardized topics
# review extracted topics and add
# you could try it with new cats or new cats + evergreen
# but probably look at new cats and human in the loop should add good new cats today to evergreen list
# new_cats = list(json.loads(response.choices[0].message.content).values())[0]
# categories = sorted(list(set(new_cats + evergreen)))
categories = sorted(CANONICAL_TOPICS)
for c in categories:
    print(c)

In [ ]:
[t for t in filtered_topics if t.lower() not in [u.lower() for u in CANONICAL_TOPICS]]

In [ ]:
# maybe try to set timeout in categorize_headline
catdict = dict()

async with aiohttp.ClientSession() as session:
    for i, row in enumerate(AIdf.itertuples()):
        tasks = []
        log(f"Categorizing headline {row.id+1} of {len(AIdf)}")
        h = row.title
        log(h)
        for c in categories:
            task = asyncio.create_task(categorize_headline(h, c, session))
            tasks.append(task)
        responses = await asyncio.gather(*tasks)
        catdict[row.id] = [item for sublist in responses for item in sublist]
        log(str(catdict[row.id]))
        
catdict

In [ ]:
topic_df["assigned_topics"] = topic_df["id"].apply(lambda id: catdict.get(id, []))
topic_df


In [ ]:
lcategories = set([c.lower() for c in categories])

In [ ]:
def clean_topics(row):
    topics = [x.title() for x in row.extracted_topics if x.lower() not in {"technology", "ai", "artificial intelligence"}]
    assigned_topics = [x.title() for x in row.assigned_topics if x.lower() in lcategories]
    combined = sorted(list(set(topics + assigned_topics)))
    combined = [s.replace("Ai", "AI") for s in combined]
    combined = [s.replace("Genai", "Gen AI") for s in combined]
    combined = [s.replace("Openai", "OpenAI") for s in combined]
    return combined

topic_df["topics"] = topic_df.apply(clean_topics, axis=1)
topic_df["topic_str"] = topic_df.apply(lambda row: ", ".join(row.topics), axis=1)
topic_df

In [ ]:
merged_df = pd.merge(AIdf, topic_df[["id", "topic_str"]], on="id", how="outer")
merged_df['title_topic_str'] = merged_df.apply(lambda row: f'{row.title} (Topics: {row.topic_str})', axis=1)
merged_df


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(merged_df.loc[merged_df["topic_str"]==""][['title']])


In [ ]:
merged_df

In [ ]:
AIdf = merged_df

### Semantic sort

In [ ]:
# use embeddings to sort headlines by semantical similarity
log(f"Fetching embeddings for {len(AIdf)} headlines")
embedding_model = 'text-embedding-3-large'
response = client.embeddings.create(input=AIdf['title_topic_str'].tolist(),
                                    model=embedding_model)
embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])

# sort of a traveling salesman sort
log(f"Sort with agglomerative cluster sort")
sorted_indices = agglomerative_cluster_sort(embedding_df)
AIdf['sort_order'] = sorted_indices

# do dimensionality reduction on embedding_df and cluster analysis
log(f"Perform dimensionality reduction")
with open("reducer.pkl", 'rb') as file:
    # Load the model from the file
    reducer = pickle.load(file)
reduced_data = reducer.transform(embedding_df)
log(f"Cluster with DBSCAN")
dbscan = DBSCAN(eps=0.4, min_samples=3)  # Adjust eps and min_samples as needed
AIdf['cluster'] = dbscan.fit_predict(reduced_data)
AIdf.loc[AIdf['cluster'] == -1, 'cluster'] = 999
    
# sort first by clusters found by DBSCAN, then by semantic ordering
AIdf = AIdf.sort_values(['cluster', 'sort_order']) \
    .reset_index(drop=True) \
    .reset_index() \
    .drop(columns=["id"]) \
    .rename(columns={'index': 'id'})

AIdf


In [ ]:
async def write_topic_name(session, topic_list_str, max_retries=3, model=LOWCOST_MODEL):

    TOPIC_WRITER_PROMPT = f"""
You are a topic writing assistant. I will provide a list of headlines with extracted topics in parentheses. 
Your task is to propose a name for a topic that very simply, clearly and accurately captures all the provided 
headlines in less than 7 words. You will output a JSON object with the key "topic_title".

Example Input:
In the latest issue of Caixins weekly magazine: CATL Bets on 'Skateboard Chassis' and Battery Swaps to Dispell Market Concerns (powered by AI) (Topics: Battery Swaps, Catl, China, Market Concerns, Skateboard Chassis)

AI, cheap EVs, future Chevy  the week (Topics: Chevy, Evs)

Electric Vehicles and AI: Driving the Consumer & World Forward (Topics: Consumer, Electric Vehicles, Technology)

Example Output:
{{"topic_title": "Electric Vehicles"}}

Task
Propose the name for the overall topic based on the following provided headlines and individual topics:

{topic_list_str}
"""

    for i in range(max_retries):
        try:
            messages=[
                      {"role": "user", "content": TOPIC_WRITER_PROMPT
                      }]

            payload = {"model":  model,
                       "response_format": {"type": "json_object"},
                       "messages": messages,
                       "temperature": 0
                       }
            response = await fetch_openai(session, payload)
            response_dict = json.loads(response["choices"][0]["message"]["content"])
            return response_dict

            break
        except Exception as exc:
            log(f"Error: {exc}")

    return {}
        

# show clusters
cluster_topics = []
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    async with aiohttp.ClientSession() as session:

        for i in range(30):
            tmpdf = AIdf.loc[AIdf['cluster']==i][["id", "title_topic_str"]]
            if len(tmpdf) ==0:
                break
            display(tmpdf)
            title_topic_str_list = ("\n\n".join(tmpdf['title_topic_str'].to_list()))
            cluster_topic = await write_topic_name(session, title_topic_str_list)
            cluster_topics.append(cluster_topic)
            print(cluster_topic)

    


In [ ]:
# we could extract top words using tfidf, something like 
# vectorizer = TfidfVectorizer(stop_words='english')
# tfidf_matrix = vectorizer.fit_transform(documents)
# feature_names = vectorizer.get_feature_names_out()
# topics = []
# for i in range(n_topics):
#     # Get the documents in this cluster
#     cluster_docs = [doc for doc, label in zip(documents, cluster_labels) if label == i]
#     cluster_metadatas = [meta for meta, label in zip(metadatas, cluster_labels) if label == i]

#     # Get the top words for this cluster based on TF-IDF scores
#     tfidf_scores = tfidf_matrix[cluster_labels == i].sum(axis=0).A1
#     top_word_indices = tfidf_scores.argsort()[-n_words_per_topic:][::-1]
#     top_words = [feature_names[index] for index in top_word_indices]


In [ ]:
cluster_topic_list = [obj['topic_title'] for obj in cluster_topics]
cluster_topic_list

In [ ]:
AIdf['cluster_name'] = AIdf['cluster'].apply(lambda i: cluster_topic_list[i] if i<len(cluster_topic_list) else "")


# Save and email headlines


In [ ]:
html_str = ""
for row in AIdf.itertuples():
    log(f"[{row.Index}. {row.title} - {row.src}]({row.actual_url})")
    html_str += f'{row.Index}.<a href="{row.url}">{row.title} - {row.src}</a><br />\n'


In [ ]:
# save headlines
with open('headlines.html', 'w') as f:
    f.write(html_str)


In [ ]:
# send mail
log("Sending headlines email")
subject = f'AI headlines {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


# Save individual pages 

In [ ]:
# fetch pages
# Create a queue for multiprocessing and populate it 
log("Queuing URLs for scraping")

queue = multiprocessing.Queue()
for row in AIdf.itertuples():
    queue.put((row.id, row.url, row.title))


In [ ]:
# scrape urls in queue asynchronously
num_browsers = 4

callable = process_url_queue_factory(queue)

log(f"fetching {len(AIdf)} pages using {num_browsers} browsers")
saved_pages = launch_drivers(num_browsers, callable)


In [ ]:
pages_df = pd.DataFrame(saved_pages)
pages_df.columns = ['id', 'url', 'title', 'path']
pages_df

In [ ]:
AIdf = pd.merge(AIdf, pages_df[["id", "path"]], on='id', how="inner")


In [ ]:
AIdf

# Summarize individual pages

In [ ]:
print(SUMMARIZE_SYSTEM_PROMPT)


In [ ]:
print(SUMMARIZE_USER_PROMPT)


In [ ]:
# Here we are fetching all at once, could be 200 summaries, so we are firing off 200 REST requests at once
# This seems like a bad idea, could loop through and fire off e.g. 10 at a time, or use queues and workers (seems pointless)
# But it works and runs fast on 3.5 and if ChatGPT doesn't like it they could throttle it

log("Starting summarize")
responses = await fetch_all_summaries(AIdf)
log(f"Received {len(responses)} summaries")
print(responses[0])


In [ ]:
# bring summaries into dict
response_dict = {}
for i, response in responses:
    try:
        response_str = response["choices"][0]["message"]["content"]
        response_dict[i] = response_str
    except Exception as exc:
        print(exc)
        
len(response_dict)

In [ ]:
# add the name of the site to the markdown
# add the name of the cluster topic title to the topics when summarizing

# keep the 
# categorize 
# use the full name of the categories

In [ ]:
AIdf['hostname']=AIdf['actual_url'].apply(lambda url: urlparse(url).netloc)
AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, ""))


In [ ]:
AIdf


In [ ]:
# make text for email and also collect data for vector store
markdown_str = ''
vectorstore_list = []
metadata_list=[]
for i, row in enumerate(AIdf.itertuples()):
    topics = []
    if row.cluster_name:
        topics.append(row.cluster_name)
    if row.topic_str:
        topics.append(row.topic_str)
    topic_str = ", ".join(topics)

    mdstr = f"[{i+1}. {row.title} - {row.site_name}]({row.actual_url})  \n\n {topic_str}  \n\n{response_dict[row.id]} \n\n"
    # simpler version for vector store
    vectorstore_list.append(f"[{row.title} - {row.site_name}]({row.actual_url})\n\nTopics: {row.topic_str} \n\n{response_dict[row.id]}\n\n")
    metadata_list.append({'id': row.id, 'title': row.title, 'url': row.actual_url, 'site': row.site_name})
    display(Markdown(mdstr))
    markdown_str += mdstr
    

In [ ]:
display(Markdown( vectorstore_list[16]))

In [ ]:
print(metadata_list[16])


In [ ]:
# Create Document objects with the paragraphs and corresponding metadata
docs = [Document(page_content=paragraph, metadata=meta) 
        for paragraph, meta in zip(vectorstore_list, metadata_list)]
len(docs)

In [ ]:
print(docs[16])


In [ ]:
# persist_directory = "/Users/drucev/projects/AInewsbot/chroma_db_openai"
try:
    del vectorstore
except Exception as e:
    log(f"{e}")

try:
    shutil.rmtree(persist_directory)
    log(f"Directory '{persist_directory}' and all its contents have been removed successfully.")
except Exception as e:
    log(f"Remove directory error: {e}")
        
embeddings_openAI = OpenAIEmbeddings(model='text-embedding-3-small')
vectorstore = Chroma.from_documents(docs, embeddings_openAI)


In [ ]:
# Perform a similarity search
query = "What is the latest with openai?"
results = vectorstore.similarity_search_with_score(query, 
                                        k=20,
                                       )  # k is the number of results to return
# Print the results
urldict = {}
for doc, score in results:
    if urldict.get(doc.metadata['url']):
        continue
    urldict[doc.metadata['url']] = 1
    if score < 1.25:
        print(f"Score:   {score}")
        print(f"Content: {doc.page_content}\n")
        print(f"Metadata: {doc.metadata}\n")
        print("---")

In [ ]:
# # or use local embeddings with sentence_transformers
# # Initialize your embedding model
# embeddings_hf = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # Create the vector store with a persist_directory
# persist_directory = "/Users/drucev/projects/AInewsbot/chroma_db_huggingface"
# vectorstore_hf = Chroma.from_documents(
#     documents=docs,
#     embedding=embeddings_hf,
#     persist_directory=persist_directory
# )

# # Perform a similarity search
# query = "What is the latest with OpenAI?"
# results = vectorstore_hf.similarity_search(query, k=10)  # k is the number of results to return

# # Print the results
# for doc in results:
#     print(f"Content: {doc.page_content}\n")
#     print(f"Metadata: {doc.metadata}\n")
#     print("---")
    


In [ ]:
# Convert Markdown to HTML
html_str = markdown.markdown(markdown_str, extensions=['extra'])
# display(HTML(html_str))


In [ ]:
# save bullets
with open('bullets.md', 'w') as f:
    f.write(markdown_str)


In [ ]:
log("Sending bullet points email")
subject = f'AI news bullets {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


# Ask ChatGPT for top categories

In [ ]:
print(TOP_CATEGORIES_PROMPT)

In [ ]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": TOP_CATEGORIES_PROMPT + markdown_str
              }],
    n=1,
    response_format={"type": "json_object"},
    temperature=0.5
)


In [ ]:
suggested_categories = list(json.loads(response.choices[0].message.content).values())[0]
suggested_categories

In [ ]:
cluster_topic_list

In [ ]:
# human category edit 
my_cats = [
    'AI Stocks',
    "Apple's Stock Price",
    'AI Policy and Regulation',
    'Google AI Overviews',
    'AI in Healthcare',
    'Samsung AI Products',
    'Future of Work and AI',
    'AI Chipmakers',
    'OpenAI Ethics Investigation',
    'Nvidia AI gold rush',
    'California AI safety bill',
    'US Senate AI ethics bill',
    'Google AI Overviews decline',
    'VerifAI biomedical project',
    'YouTube Music AI features',
    'Cybersecurity risks in LLMs',
    'Disney AI music partnership',
    'AI decision-making model',
    'AI in chronic disease management',
    'AI-driven copper discovery',
    'AI-powered grocery shopping',
    'AI in gaming industry',
    'AI ethics council',
    'AI in financial literacy'    
    
]

In [ ]:
md_str = ""
doc_list = []
similarity_cutoff = 1.25
for cat in my_cats:
    docstr = f"# {cat} \n\n"
    # Perform a similarity search
    results = vectorstore.similarity_search_with_score(cat, 
                                                       k=10,
                                                      )
    if results:
        # Print the results
        urldict = {}
        for doc, score in results:
            if urldict.get(doc.metadata['url']):
                continue
            urldict[doc.metadata['url']] = 1    
            if score > similarity_cutoff:
                break
            docstr += f"{doc.page_content}\n"
        doc_list.append(docstr)
        md_str += docstr
        
        
display(Markdown(md_str))
            


In [ ]:
mail_md_str = ""

for current_topic, cat in enumerate(my_cats):

    section_prompt = f"""
You are an advanced summarization assistant, a sophisticated AI system
designed to write a compelling summary of news input.

Input:
I will provide a markdown list of today's news articles on the topic: {my_cats[current_topic]}.
The input will be in the format
[Site-name-s1](url-s1)
Story-Title-s1

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

[Site-name-s2](url-s2)
Story-Title-s2

Topics: s2-topic1, s2-topic2, s2-topic3

- s2-bullet-point-1
- s2-bullet-point-2
- s2-bullet-point-3

Instructions:

Read the input closely.
USE ONLY INFORMATION PROVIDED IN THE INPUT.
Provide the most significant facts without commentary or elaboration.
Write an engaging summary consisting of a title and at least 1 and no more than 5 bullet points.
Use as few bullet points as you need to provide the most significant facts.
Each bullet should contain one sentence with one link.
Each bullet should not repeat points or information from previous bullet points.
DO NOT REPEAT LINKS FROM PREVIOUS BULLET POINTS.
Write in the professional but engaging, narrative style of a tech reporter for a national publication.
Be balanced, professional, informative, providing accurate, clear, concise summaries in a respectful neutral tone.

Please check carefully that you only use information provided in the following input, and that any bullet point
does not repeat information or links prevously provided.

Example Output Format Template (EXAMPLE ONLY, DO NOT OUTPUT THIS TEMPLATE):

# Engaging title

- bullet point a - [site name a](site url a)
- bullet point b - [site name b ](site url b)

Input:

{doc_list[current_topic]}
"""

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
                  {"role": "user", "content": section_prompt
                  }],
        n=1,   
        temperature=0.2
    )

    response_str = response.choices[0].message.content
    response_str = response_str.replace("$", "\\$")
    mail_md_str += response_str + " \n\n"
    display(Markdown(response_str))


In [ ]:
print(mail_md_str)

In [ ]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(mail_md_str, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


# Final Summary

In [ ]:
markdown_str = ''
for i, row in enumerate(AIdf.itertuples()):
    mdstr = f"[{i+1}. {row.title} - {row.site_name}]({row.url})  \n\n"
#     if 0 < len(catdict[row.id]) < 11 :
#         topicstr = ", ".join(catdict[row.id])
#         mdstr += f"Topics: {topicstr}\n\n"
    mdstr += f"Topics: {row.topic_str}\n\n"
    mdstr += f"{response_dict[row.id]} \n\n"
    display(Markdown(mdstr.replace('$', '\\$')))
    markdown_str += mdstr
    

In [ ]:
# clean up the list of topics in human in the loop workflow
# loop though each topic and summarize, then 
# then combine the summaries for a final prompt

In [ ]:
print(FINAL_SUMMARY_PROMPT)


In [ ]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": FINAL_SUMMARY_PROMPT + markdown_str
              }],
    n=1,   
    temperature=0.5
)

response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


In [ ]:
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


In [ ]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(response_str, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


In [ ]:
log("Finished")


In [ ]:
"""You will act like a professional editor with expertise in content optimization.
You are skilled at refining and enhancing written materials, specializing in
ensuring clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Edit the markdown newsletter provided below by removing any redundant
sentences or bullet points that restate previous points and contain the same link.
Leave intact bullet points that are unique and provide distinct information.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Identify sentences and bullet points that repeat information or provide identical links.
Remove all redundant sentences and bullet points that do not contribute new information or unique links.
Ensure that the remaining content flows logically and maintains the intended message and tone of the newsletter.
Double-check the final edited version for any inconsistencies or errors introduced during the editing process.
Take a deep breath and work on this problem step-by-step.
"""

In [ ]:
"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Review the markdown newsletter provided below and advise on ways to improve it.
Note any links which are repeated, any sections which are similar and could be combined,
and any copy edits. You will only provide suggestions, and not rewrite the copy.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Identify sentences and bullet points that repeat information and provide identical links and should be removed.
Identify any sections which could be combined because they contain similar but not identical content.
Suggest improvements to any sections which are not clear, concise, and coherent.
Take a deep breath and work on this problem step-by-step.
"""

In [ ]:
display(Markdown(mail_md_str.replace("$", "\\$")))


In [134]:
edit_prompt1 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Review the markdown newsletter provided below.
It consists of a series of sections, each of which contains several bullet points.
For each section, review each bullet point and advise if it should be moved to a different section.
You will only provide suggestions, and not rewrite the newsletter or provide other comments except
instructions regarding moving bullet points between sections.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Note the titles of the various sections.
Identify sentences and bullet points that should be moved to a different section. Write the
bullet point and the section in should be moved to.
If no bullet points should be moved for a given section, state that no action is required for that section.

Check carefully to make sure all similar bullet points end up grouped together in the same section.

Take a deep breath and work on this problem step-by-step.

Newsletter to edit: 
{mail_md_str}
"""
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt1
              }],
    n=1,   
    temperature=0.2
)

response_str1 = response.choices[0].message.content
display(Markdown(response_str1.replace("$", "\\$")))


2024-07-16 00:05:04,238 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI Stocks Surge Amid Technological Advancements

- Emerging-market equity investors are finding new opportunities in AI beyond US tech giants, with improved governance in markets like Japan and Korea - [AllianceBernstein](https://www.alliancebernstein.com/nz/en/institutions/insights/investment-insights/is-ai-a-new-engine-for-emerging-market-equity-investors.html).
- Despite market volatility, AI remains a dominant investment theme with significant long-term growth potential, impacting productivity gains globally - [Invesco](https://www.invesco.com/hk/en/investment-ideas/quick-takes/ai-series-how-ai-remains-dominant-investment-theme.html).
- Nvidia and Super Micro Computer have seen significant gains, while Alibaba and Innodata are poised for substantial growth in the AI sector - [Yahoo Finance](https://finance.yahoo.com/news/2-artificial-intelligence-ai-stocks-092500064.html).
- Vistra Corp. has outperformed with a 1-year total return of over 240%, driven by AI data center power demand and integration of Energy Harbor - [Seeking Alpha](https://seekingalpha.com/article/4704133-vistra-ai-game-changer-is-just-getting-started).
- Arm Holdings and STMicroelectronics are set to benefit from the AI trend, with advancements in AI and 5G technologies - [Barchart](https://www.barchart.com/story/news/27341677/missed-nvidia-2-semiconductor-stocks-that-could-gain-from-the-ai-frenzy).
- Broadcom is considered a better AI chip buy over Marvell Technology due to its diversified business and stable profits - [The Motley Fool](https://www.fool.com/investing/2024/07/15/better-ai-chip-stock-broadcom-vs-marvell-technolog/).

**Move to "Apple's Stock Price Soars on AI Optimism":**
- Apple's stock rose after being named a "top pick" by Morgan Stanley for its AI efforts, with projections of nearly 500 million iPhone sales over the next two years - [Reddit](https://www.reddit.com/r/technology/comments/1e3u0w3/apple_gains_after_morgan_stanley_calls_stock_top/).
- Apple stock hits a record high with analysts optimistic about generative AI on iPhones and iPads, projecting up to 500 million iPhone sales in the next two years - [Investopedia](https://www.investopedia.com/apple-stock-price-record-high-analysts-targets-ai-8678016).
- Analysts predict a significant AI-driven iPhone upgrade cycle, with potential shipments of nearly 500 million iPhones in fiscal 2025 and 2026 - [CNBC](https://www.cnbc.com/2024/07/15/the-ai-driven-iphone-upgrade-cycle-will-be-huge-two-more-wall-street-firms-agree.html).

# Apple's Stock Price Soars on AI Optimism

- Apple stock hits a record high after receiving price upgrades based on analysts' optimism about generative AI on iPhones and iPads - [Investopedia](https://www.investopedia.com/apple-stock-price-record-high-analysts-targets-ai-8678016).
- Apple's stock rose after being designated as a "top pick" by Morgan Stanley for its AI efforts, with a raised price target boosting device sales - [Reddit](https://www.reddit.com/r/technology/comments/1e3u0w3/apple_gains_after_morgan_stanley_calls_stock_top/).
- Analysts predict a significant iPhone upgrade cycle, with the potential for Apple to ship nearly 500 million iPhones in fiscal 2025 and 2026 combined - [CNBC](https://www.cnbc.com/2024/07/15/the-ai-driven-iphone-upgrade-cycle-will-be-huge-two-more-wall-street-firms-agree.html).

**Move to "AI Stocks Surge Amid Technological Advancements":**
- Tech giants like Apple, Microsoft, and Nvidia are driving the stock market surge, with a focus on AI technology - [Forbes](https://www.forbes.com/sites/cio/2024/07/11/as-valuations-around-ai-skyrocket-how-should-cios-look-at-solutions/).

# Key Developments in AI Policy and Regulation

- The New York Department of Financial Services (NYDFS) mandates that insurance companies' use of AI must not discriminate against consumers - [PYMNTS](https://www.pymnts.com/insurance/2024/nydfs-issues-guidance-for-insurance-companies-use-of-ai/).
- California's AI bill SB-1047, which aims to regulate powerful AI models for safety, is sparking a fierce debate between proponents and opponents - [Fortune](https://fortune.com/2024/07/15/california-ai-bill-sb-1047-fierce-debate-regulation-safety/).
- Yann LeCun opposes California's proposed AI Safety Bill, warning that it could stifle innovation in AI research - [Analytics India Magazine](https://analyticsindiamag.com/ai-news-updates/yann-lecun-urges-signing-letter-to-block-regulation-for-ai-research/).
- OpenAI whistleblowers have filed a complaint with the SEC regarding the company's non-disclosure agreements that limit employees from discussing AI risks - [Boston 25 News](https://www.boston25news.com/news/openai/XENZN4NAQBEHVNVXCDMPCY3RXQ/).
- The US Senate introduced the "Content Origin Protection and Integrity from Edited and Deepfaked Media Act" (COPIED Act) to regulate the use of AI-generated content and deepfake technology - [TechSpot](https://www.techspot.com/news/103820-us-senate-introduces-ground-breaking-bill-setup-legal.html).

**Move to "AI in Healthcare: Today's Innovations and Challenges":**
- NYDFS mandates non-discriminatory AI use in insurance, emphasizing transparency and fairness - [PYMNTS](https://www.pymnts.com/insurance/2024/nydfs-issues-guidance-for-insurance-companies-use-of-ai/).

# Google AI Overviews: Current Trends and Developments

- Google has significantly reduced the display of AI Overviews in Search, now appearing in less than 10% of search queries - [9to5Google](https://9to5google.com/2024/07/15/google-ai-overviews-decline-report-june-2024/).
- Google AI Overviews now appear less than 7% of the time, with significant drops in visibility on education, entertainment, and ecommerce queries - [Search Engine Land](https://searchengineland.com/google-ai-overviews-visibility-new-low-444048).
- Google has introduced AI Overviews, a new version of its search engine, causing concerns among businesses about its impact on web traffic, advertising revenue, and visibility for smaller businesses online - [Tech in Asia](https://www.techinasia.com/error-404-web-killed-googles-ai-search).
- Google has launched Vids, an AI-powered video maker, in Workspace Labs, allowing users to create presentation videos by combining docs, slides, voiceovers, and video recordings - [The Verge](https://www.theverge.com/2024/7/15/24199063/google-vids-gemini-ai-app-workspace-labs-available).

**Move to "AI Stocks Surge Amid Technological Advancements":**
- Lee Se-Dol, a top Go player, was defeated by an AI program in 2016, leading to his retirement in 2019 - [Business Insider](https://www.businessinsider.in/international/news/one-of-the-worlds-greatest-go-players-who-was-defeated-by-ai-warns-that-the-technology-may-not-come-with-a-happy-ending/articleshow/111668042.cms).

# AI in Healthcare: Today's Innovations and Challenges

- Kaiser Permanente uses advanced AI for early intervention in patient care, enhancing workflow and saving lives - [VentureBeat](https://venturebeat.com/ai/how-kaiser-permanente-is-using-gen-ai-to-paradoxically-make-care-more-human-again/)
- Cambridge scientists' AI tool predicts Alzheimer's progression with higher accuracy than current clinical markers - [Medical Xpress](https://medicalxpress.com/news/2024-07-artificial-intelligence-outperforms-clinical-alzheimer.html)
- Thrive AI Health's personalized AI health coach aims to address chronic conditions through behavior change - [Fortune](https://fortune.com/well/2024/07/08/thrive-ai-health-coach-sam-altman-arianna-huffington/)
- Fast Forward accelerator supports AI startups tackling global issues, focusing on ethical AI deployment - [Triple Pundit](https://www.triplepundit.com/story/2024/artificial-intelligence-Fast-Forward/805751)

**Move to "Key Developments in AI Policy and Regulation":**
- NYDFS mandates non-discriminatory AI use in insurance, emphasizing transparency and fairness - [PYMNTS](https://www.pymnts.com/insurance/2024/nydfs-issues-guidance-for-insurance-companies-use-of-ai/)

# Samsung AI Innovations: A New Era in Tech

- Samsung introduces Galaxy Ring, a smart ring with AI capabilities for health and fitness monitoring, featuring a lightweight titanium frame and up to seven days of battery life - [Women's Wear Daily](https://wwd.com/business-news/technology/samsungs-galaxy-ring-ai-wearable-jewelry-1236492272/)
- Samsung confirms older Galaxy devices, including the S22, S23, and S24 series, will receive the One UI 6.1.1 update with new AI features like Generative AI for camera enhancements - [Android Central](https://www.androidcentral.com/apps-software/samsung-one-ui-6-1-1-devices-confirmed)
- One UI 6.1.1 introduces AI suggested replies for quick responses on Galaxy Z Flip 6 and One UI Watch 6, ensuring privacy by processing messages on-device - [Android Authority](https://www.androidauthority.com/samsung-suggested-replies-one-ui-3460745/)
- Samsung leaves Galaxy S21 out of the One UI 6.1.1 update, focusing AI enhancements on newer models like the S22 and Z Fold series due to potential processing power limitations - [Phone Arena](https://www.phonearena.com/news/Samsung-leaves-Galaxy-S21-out-of-the-One-UI-6.1.1-update-AI-party_id160416)
- The Samsung Galaxy Ring has sold out on Samsung's online store, with limited supplies at third-party retailers like Best Buy, due to its positive reviews and lack of a subscription model - [TechRadar](https://www.techradar.com/health-fitness/its-not-you-galaxy-ring-just-sold-out-from-samsung-but-weve-found-another-way-for-you-to-put-a-ring-on-it)

No action is required for this section.

# The Future of Work and AI: Transformative Trends

- AI programs like ChatGPT and Microsoft Copilot are enhancing productivity by automating tasks and improving workflows - [SoGlos](https://www.soglos.com/interview/business/ai-is-going-to-become-a-game-changer-in-the-workplace-says-leading-cheltenham-tech-firm/23158/).
- Businesses are evolving towards neural organizations, emphasizing interconnectedness for innovation and speed - [McKinsey & Company](https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/gen-ai-and-beyond-where-else-to-focus-now).
- 77% of professionals believe AI will significantly impact their work in the next five years, though trust in AI for autonomous decision-making remains low - [TechRepublic](https://www.techrepublic.com/article/thomson-reuters-ai-report/).
- The food industry is investing heavily in AI and automation to improve service and reduce costs, with companies like McDonald’s leading the way - [CNBC](https://www.cnbc.com/2024/07/15/how-ai-and-automation-will-reshape-grocery-stores-and-fast-food-chains.html).
- AI advancements in finance are seen as a potential solution to banking discrimination and job displacement concerns - [Newsweek](https://www.newsweek.com/sam-altman-ai-ethics-minimize-discrimination-banking-create-jobs-1923321).
- Fast Forward's AI for Humanity initiative supports startups using AI to address global issues such as poverty and healthcare inequity - [Triple Pundit](https://www.triplepundit.com/story/2024/artificial-intelligence-Fast-Forward/805751).
- Instacart's AI-powered smart carts are being introduced to more grocery stores, enhancing shopping experiences with real-time recommendations and gamification - [Fortune](https://fortune.com/2024/07/08/instacart-ai-powered-caper-carts-artificial-intelligence-gamification-grocery-stores/).

**Move to "AI Stocks Surge Amid Technological Advancements":**
- Tech giants like Apple, Microsoft, and Nvidia are driving a stock market surge with their focus on AI technology - [Forbes](https://www.forbes.com/sites/cio/2024/07/11/as-valuations-around-ai-skyrocket-how-should-cios-look-at-solutions/).

# AI Chipmakers: Market Trends and Innovations

- Companies are heavily investing in AI accelerators, with forecasts predicting the datacenter AI accelerator market to reach billions by 2027 - [The Next Platform](https://www.nextplatform.com/2024/07/15/ongoing-saga-how-much-money-will-be-spent-on-ai-chips/).
- Arm Holdings and STMicroelectronics are poised to benefit from the AI trend, with ARM showing remarkable revenue growth - [Barchart](https://www.barchart.com/story/news/27341677/missed-nvidia-2-semiconductor-stocks-that-could-gain-from-the-ai-frenzy).
- Broadcom is considered a better AI chip buy than Marvell Technology due to its diversified business and stable profits - [The Motley Fool](https://www.fool.com/investing/2024/07/15/better-ai-chip-stock-broadcom-vs-marvell-technolog/).
- Nvidia's stock gains as analysts remain optimistic about its earnings, driven by strong demand for data center AI systems - [MarketWatch](https://www.marketwatch.com/story/nvidias-stock-gains-as-an-analyst-throws-cold-water-on-this-bearish-concern-2ced7ff2).
- AMD has launched the Ryzen AI 300 series, featuring new Zen 5 cores and RDNA 3.5 GPU, enhancing AI capabilities - [Serve The Home](https://www.servethehome.com/amd-ryzen-ai-300-series-launched/).

No action is required for this section.

# OpenAI Ethics Investigation: Key Developments

- OpenAI whistleblowers are urging the SEC to investigate the company for alleged violations related to prohibiting whistleblowing about safety risks - [Mashable](https://sea.mashable.com/tech/33422/openai-whistleblowers-call-on-sec-to-investigate-the-ai-company).
- U.S. Sen. Chuck Grassley highlighted concerns about OpenAI's policies silencing whistleblowers and emphasized the need for change in their NDA practices - [Boston 25 News](https://www.boston25news.com/news/openai/XENZN4NAQBEHVNVXCDMPCY3RXQ/).
- OpenAI is reportedly progressing rapidly towards human-level reasoning, unveiling a system to track progress towards A.G.I., with plans for models to act as agents, innovators, and organizations - [Observer](https://observer.com/2024/07/openai-employees-concerns-strawberry-ai-project/).
- An AI ethics council, co-chaired by Sam Altman of OpenAI and John Hope Bryant, aims to ensure underserved communities are not left behind by AI - [AJC](https://www.ajc.com/news/business/ai-ethics-council-hosts-inaugural-meeting-in-atlanta-looks-to-expand/QCAFZQUALJAJHJUJFZUVSO3XSA/).

**Move to "Key Developments in AI Policy and Regulation":**
- The US Senate introduced the "Content Origin Protection and Integrity from Edited and Deepfaked Media Act" (COPIED Act) to address concerns over unethical use of AI-generated content and deepfake technology - [TechSpot](https://www.techspot.com/news/103820-us-senate-introduces-ground-breaking-bill-setup-legal.html).

# Nvidia AI Gold Rush: Key Developments

- Providers of AI infrastructure are benefiting from the growing demand in the Nvidia-led AI gold rush, with hyperscale data centers projected to grow to \\$262 billion by 2032 - [MarketWatch](https://www.marketwatch.com/story/the-nvidia-led-ai-gold-rush-is-paydirt-for-these-5-data-center-growth-stocks-8f30534c).
- Wall Street analysts remain bullish on Nvidia stock, despite investor caution, with expectations of strong fiscal Q2 results driven by AI system demand - [Investor's Business Daily](https://www.investors.com/news/technology/nvidia-stock-nvda-analysts-bullish/).
- FlashAttention-3 optimizes attention computation for large language models using Nvidia Hopper GPUs, achieving significant speedups and efficiency improvements - [VentureBeat](https://venturebeat.com/ai/flashattention-3-unleashes-the-power-of-h100-gpus-for-llms/).
- TD Cowen analyst predicts Nvidia will outperform expectations due to sustained demand for Hopper-based AI systems, despite concerns about slowing AI hardware spending - [MarketWatch](https://www.marketwatch.com/story/nvidias-stock-gains-as-an-analyst-throws-cold-water-on-this-bearish-concern-2ced7ff2).
- Arrow Electronics' semi-autonomous motorcar project utilizes Nvidia modules for hands-free driving at 200 mph, showcasing advancements in edge AI and autonomous vehicle technology - [EE Times](https://www.eetimes.com/the-ultimate-edge-ai-hands-free-driving-at-200-mph/).

No action is required for this section.

# California AI Safety Bill Sparks Debate and Controversy

- California's AI bill SB-1047 is igniting a fierce debate over AI regulation, with proponents advocating for safety and ethical standards, while opponents fear stifling innovation and consolidation of power among tech giants - [Fortune](https://fortune.com/2024/07/15/california-ai-bill-sb-1047-fierce-debate-regulation-safety/).
- The bill mandates strict guidelines for AI developers, including safety measures, certifications, and incident reporting, enforced by the Attorney General and Labor Commissioner - [The National Law Review](https://natlawreview.com/article/california-dreamin-will-sb-1047-curb-ai-innovation).
- Yann LeCun opposes the bill, warning it could hinder AI research, while figures like Geoffrey Hinton and Yoshua Bengio support it for balancing safety and innovation - [Analytics India Magazine](https://analyticsindiamag.com/ai-news-updates/yann-lecun-urges-signing-letter-to-block-regulation-for-ai-research/).

No action is required for this section.

# US Senate AI Ethics Bill: Key Developments

- The US Senate introduced the "Content Origin Protection

In [135]:
edit_prompt2 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Below are editing instructions followed by a markdown newsletter.
Carefully review the editing instructions and the markdown newsletter provided below.
The newsletter consists of a series of sections, each of which contains several bullet points.
Move bullet points according to the editing instructions below from one section to another 
If there is no change to a specific section, include it unchanged in the response as it appears in the input.
Respond with the updated newsletter in markdown format.

Editing instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Note the titles of the various sections. Then make only the following changes:
{response_str1}

Newsletter to edit: 
{mail_md_str}

"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt2
              }],
    n=1,   
    temperature=0.2
)
response_str2 = response.choices[0].message.content
display(Markdown(response_str2.replace("$", "\\$")))


2024-07-16 00:06:03,377 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


```markdown
# AI Stocks Surge Amid Technological Advancements

- Emerging-market equity investors are finding new opportunities in AI beyond US tech giants, with improved governance in markets like Japan and Korea - [AllianceBernstein](https://www.alliancebernstein.com/nz/en/institutions/insights/investment-insights/is-ai-a-new-engine-for-emerging-market-equity-investors.html).
- Despite market volatility, AI remains a dominant investment theme with significant long-term growth potential, impacting productivity gains globally - [Invesco](https://www.invesco.com/hk/en/investment-ideas/quick-takes/ai-series-how-ai-remains-dominant-investment-theme.html).
- Nvidia and Super Micro Computer have seen significant gains, while Alibaba and Innodata are poised for substantial growth in the AI sector - [Yahoo Finance](https://finance.yahoo.com/news/2-artificial-intelligence-ai-stocks-092500064.html).
- Apple's stock rose after being named a "top pick" by Morgan Stanley for its AI efforts, with projections of nearly 500 million iPhone sales over the next two years - [Reddit](https://www.reddit.com/r/technology/comments/1e3u0w3/apple_gains_after_morgan_stanley_calls_stock_top/).
- Vistra Corp. has outperformed with a 1-year total return of over 240%, driven by AI data center power demand and integration of Energy Harbor - [Seeking Alpha](https://seekingalpha.com/article/4704133-vistra-ai-game-changer-is-just-getting-started).
- Arm Holdings and STMicroelectronics are set to benefit from the AI trend, with advancements in AI and 5G technologies - [Barchart](https://www.barchart.com/story/news/27341677/missed-nvidia-2-semiconductor-stocks-that-could-gain-from-the-ai-frenzy).
- Apple stock hits a record high with analysts optimistic about generative AI on iPhones and iPads, projecting up to 500 million iPhone sales in the next two years - [Investopedia](https://www.investopedia.com/apple-stock-price-record-high-analysts-targets-ai-8678016).
- Broadcom is considered a better AI chip buy over Marvell Technology due to its diversified business and stable profits - [The Motley Fool](https://www.fool.com/investing/2024/07/15/better-ai-chip-stock-broadcom-vs-marvell-technolog/).
- Analysts predict a significant AI-driven iPhone upgrade cycle, with potential shipments of nearly 500 million iPhones in fiscal 2025 and 2026 - [CNBC](https://www.cnbc.com/2024/07/15/the-ai-driven-iphone-upgrade-cycle-will-be-huge-two-more-wall-street-firms-agree.html).
- Tech giants like Apple, Microsoft, and Nvidia are driving the stock market surge, with a focus on AI technology - [Forbes](https://www.forbes.com/sites/cio/2024/07/11/as-valuations-around-ai-skyrocket-how-should-cios-look-at-solutions/).
- Lee Se-Dol, a top Go player, was defeated by an AI program in 2016, leading to his retirement in 2019 - [Business Insider](https://www.businessinsider.in/international/news/one-of-the-worlds-greatest-go-players-who-was-defeated-by-ai-warns-that-the-technology-may-not-come-with-a-happy-ending/articleshow/111668042.cms).

# Apple's Stock Price Soars on AI Optimism

- Apple stock hits a record high after receiving price upgrades based on analysts' optimism about generative AI on iPhones and iPads - [Investopedia](https://www.investopedia.com/apple-stock-price-record-high-analysts-targets-ai-8678016).
- Apple's stock rose after being designated as a "top pick" by Morgan Stanley for its AI efforts, with a raised price target boosting device sales - [Reddit](https://www.reddit.com/r/technology/comments/1e3u0w3/apple_gains_after_morgan_stanley_calls_stock_top/).
- Analysts predict a significant iPhone upgrade cycle, with the potential for Apple to ship nearly 500 million iPhones in fiscal 2025 and 2026 combined - [CNBC](https://www.cnbc.com/2024/07/15/the-ai-driven-iphone-upgrade-cycle-will-be-huge-two-more-wall-street-firms-agree.html).
- Tech giants like Apple, Microsoft, and Nvidia are driving the stock market surge, with a focus on AI technology - [Forbes](https://www.forbes.com/sites/cio/2024/07/11/as-valuations-around-ai-skyrocket-how-should-cios-look-at-solutions/).

# Key Developments in AI Policy and Regulation

- The New York Department of Financial Services (NYDFS) mandates that insurance companies' use of AI must not discriminate against consumers - [PYMNTS](https://www.pymnts.com/insurance/2024/nydfs-issues-guidance-for-insurance-companies-use-of-ai/).
- California's AI bill SB-1047, which aims to regulate powerful AI models for safety, is sparking a fierce debate between proponents and opponents - [Fortune](https://fortune.com/2024/07/15/california-ai-bill-sb-1047-fierce-debate-regulation-safety/).
- Yann LeCun opposes California's proposed AI Safety Bill, warning that it could stifle innovation in AI research - [Analytics India Magazine](https://analyticsindiamag.com/ai-news-updates/yann-lecun-urges-signing-letter-to-block-regulation-for-ai-research/).
- OpenAI whistleblowers have filed a complaint with the SEC regarding the company's non-disclosure agreements that limit employees from discussing AI risks - [Boston 25 News](https://www.boston25news.com/news/openai/XENZN4NAQBEHVNVXCDMPCY3RXQ/).
- The US Senate introduced the "Content Origin Protection and Integrity from Edited and Deepfaked Media Act" (COPIED Act) to regulate the use of AI-generated content and deepfake technology - [TechSpot](https://www.techspot.com/news/103820-us-senate-introduces-ground-breaking-bill-setup-legal.html).

# Google AI Overviews: Current Trends and Developments

- Google has significantly reduced the display of AI Overviews in Search, now appearing in less than 10% of search queries - [9to5Google](https://9to5google.com/2024/07/15/google-ai-overviews-decline-report-june-2024/).
- Google AI Overviews now appear less than 7% of the time, with significant drops in visibility on education, entertainment, and ecommerce queries - [Search Engine Land](https://searchengineland.com/google-ai-overviews-visibility-new-low-444048).
- Google has introduced AI Overviews, a new version of its search engine, causing concerns among businesses about its impact on web traffic, advertising revenue, and visibility for smaller businesses online - [Tech in Asia](https://www.techinasia.com/error-404-web-killed-googles-ai-search).
- Google has launched Vids, an AI-powered video maker, in Workspace Labs, allowing users to create presentation videos by combining docs, slides, voiceovers, and video recordings - [The Verge](https://www.theverge.com/2024/7/15/24199063/google-vids-gemini-ai-app-workspace-labs-available).

# AI in Healthcare: Today's Innovations and Challenges

- Kaiser Permanente uses advanced AI for early intervention in patient care, enhancing workflow and saving lives - [VentureBeat](https://venturebeat.com/ai/how-kaiser-permanente-is-using-gen-ai-to-paradoxically-make-care-more-human-again/)
- Cambridge scientists' AI tool predicts Alzheimer's progression with higher accuracy than current clinical markers - [Medical Xpress](https://medicalxpress.com/news/2024-07-artificial-intelligence-outperforms-clinical-alzheimer.html)
- Thrive AI Health's personalized AI health coach aims to address chronic conditions through behavior change - [Fortune](https://fortune.com/well/2024/07/08/thrive-ai-health-coach-sam-altman-arianna-huffington/)
- NYDFS mandates non-discriminatory AI use in insurance, emphasizing transparency and fairness - [PYMNTS](https://www.pymnts.com/insurance/2024/nydfs-issues-guidance-for-insurance-companies-use-of-ai/)
- Fast Forward accelerator supports AI startups tackling global issues, focusing on ethical AI deployment - [Triple Pundit](https://www.triplepundit.com/story/2024/artificial-intelligence-Fast-Forward/805751)

# Samsung AI Innovations: A New Era in Tech

- Samsung introduces Galaxy Ring, a smart ring with AI capabilities for health and fitness monitoring, featuring a lightweight titanium frame and up to seven days of battery life - [Women's Wear Daily](https://wwd.com/business-news/technology/samsungs-galaxy-ring-ai-wearable-jewelry-1236492272/)
- Samsung confirms older Galaxy devices, including the S22, S23, and S24 series, will receive the One UI 6.1.1 update with new AI features like Generative AI for camera enhancements - [Android Central](https://www.androidcentral.com/apps-software/samsung-one-ui-6-1-1-devices-confirmed)
- One UI 6.1.1 introduces AI suggested replies for quick responses on Galaxy Z Flip 6 and One UI Watch 6, ensuring privacy by processing messages on-device - [Android Authority](https://www.androidauthority.com/samsung-suggested-replies-one-ui-3460745/)
- Samsung leaves Galaxy S21 out of the One UI 6.1.1 update, focusing AI enhancements on newer models like the S22 and Z Fold series due to potential processing power limitations - [Phone Arena](https://www.phonearena.com/news/Samsung-leaves-Galaxy-S21-out-of-the-One-UI-6.1.1-update-AI-party_id160416)
- The Samsung Galaxy Ring has sold out on Samsung's online store, with limited supplies at third-party retailers like Best Buy, due to its positive reviews and lack of a subscription model - [TechRadar](https://www.techradar.com/health-fitness/its-not-you-galaxy-ring-just-sold-out-from-samsung-but-weve-found-another-way-for-you-to-put-a-ring-on-it)

# The Future of Work and AI: Transformative Trends

- AI programs like ChatGPT and Microsoft Copilot are enhancing productivity by automating tasks and improving workflows - [SoGlos](https://www.soglos.com/interview/business/ai-is-going-to-become-a-game-changer-in-the-workplace-says-leading-cheltenham-tech-firm/23158/).
- Businesses are evolving towards neural organizations, emphasizing interconnectedness for innovation and speed - [McKinsey & Company](https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/gen-ai-and-beyond-where-else-to-focus-now).
- 77% of professionals believe AI will significantly impact their work in the next five years, though trust in AI for autonomous decision-making remains low - [TechRepublic](https://www.techrepublic.com/article/thomson-reuters-ai-report/).
- The food industry is investing heavily in AI and automation to improve service and reduce costs, with companies like McDonald’s leading the way - [CNBC](https://www.cnbc.com/2024/07/15/how-ai-and-automation-will-reshape-grocery-stores-and-fast-food-chains.html).
- AI advancements in finance are seen as a potential solution to banking discrimination and job displacement concerns - [Newsweek](https://www.newsweek.com/sam-altman-ai-ethics-minimize-discrimination-banking-create-jobs-1923321).
- Tech giants like Apple, Microsoft, and Nvidia are driving a stock market surge with their focus on AI technology - [Forbes](https://www.forbes.com/sites/cio/2024/07/11/as-valuations-around-ai-skyrocket-how-should-cios-look-at-solutions/).
- Fast Forward's AI for Humanity initiative supports startups using AI to address global issues such as poverty and healthcare inequity - [Triple Pundit](https://www.triplepundit.com/story/2024/artificial-intelligence-Fast-Forward/805751).
- Instacart's AI-powered smart carts are being introduced to more grocery stores, enhancing shopping experiences with real-time recommendations and gamification - [Fortune](https://fortune.com/2024/07/08/instacart-ai-powered-caper-carts-artificial-intelligence-gamification-grocery-stores/).

# AI Chipmakers: Market Trends and Innovations

- Companies are heavily investing in AI accelerators, with forecasts predicting the datacenter AI accelerator market to reach billions by 2027 - [The Next Platform](https://www.nextplatform.com/2024/07/15/ongoing-saga-how-much-money-will-be-spent-on-ai-chips/).
- Arm Holdings and STMicroelectronics are poised to benefit from the AI trend, with ARM showing remarkable revenue growth - [Barchart](https://www.barchart.com/story/news/27341677/missed-nvidia-2-semiconductor-stocks-that-could-gain-from-the-ai-frenzy).
- Broadcom is considered a better AI chip buy than Marvell Technology due to its diversified business and stable profits - [The Motley Fool](https://www.fool.com/investing/2024/07/15/better-ai-chip-stock-broadcom-vs-marvell-technolog/).
- Nvidia's stock gains as analysts remain optimistic about its earnings, driven by strong demand for data center AI systems - [MarketWatch](https://www.marketwatch.com/story/nvidias-stock-gains-as-an-analyst-throws-cold-water-on-this-bearish-concern-2ced7ff2).
- AMD has launched the Ryzen AI 300 series, featuring new Zen 5 cores and RDNA 3.5 GPU, enhancing AI capabilities - [Serve The Home](https://www.servethehome.com/amd-ryzen-ai-300-series-launched/).

# OpenAI Ethics Investigation: Key Developments

- OpenAI whistleblowers are urging the SEC to investigate the company for alleged violations related to prohibiting whistleblowing about safety risks - [Mashable](https://sea.mashable.com/tech/33422/openai-whistleblowers-call-on-sec-to-investigate-the-ai-company).
- U.S. Sen. Chuck Grassley highlighted concerns about OpenAI's policies silencing whistleblowers and emphasized the need for change in their NDA practices - [Boston 25 News](https://www.boston25news.com/news/openai/XENZN4NAQBEHVNVXCDMPCY3RXQ/).
- OpenAI is reportedly progressing rapidly towards human-level reasoning, unveiling a system to track progress towards A.G.I., with plans for models to act as agents, innovators, and organizations - [Observer](https://observer.com/2024/07/openai-employees-concerns-strawberry-ai-project/).
- An AI ethics council, co-chaired by Sam Altman of OpenAI and John Hope Bryant, aims to ensure underserved communities are not left behind by AI - [AJC](https://www.ajc.com/news/business/ai-ethics-council-hosts-inaugural-meeting-in-atlanta-looks-to-expand/QCAFZQUALJAJHJUJFZUVSO3XSA/).
- The US Senate introduced the "Content Origin Protection and Integrity from Edited and Deepfaked Media Act" (COPIED Act) to address concerns over unethical use of AI-generated content and deepfake technology - [TechSpot](https://www.techspot.com/news/103820-us-senate-introduces-ground-breaking-bill-setup-legal.html).

# Nvidia AI Gold Rush: Key Developments

- Providers of AI infrastructure are benefiting from the growing demand in the Nvidia-led AI gold rush, with hyperscale data centers projected to grow to \\$262 billion by 2032 - [MarketWatch](https://www.marketwatch.com/story/the-nvidia-led-ai-gold-rush-is-paydirt-for-these-5-data-center-growth-stocks-8f30534c).
- Wall Street analysts remain bullish on Nvidia stock, despite investor caution, with expectations of strong fiscal Q2 results driven by AI system demand - [Investor's Business Daily](https://www.investors.com/news/technology/nvidia-stock-nvda-analysts-bullish/).
- FlashAttention-3 optimizes attention computation for large language models using Nvidia Hopper GPUs, achieving significant speedups and efficiency improvements - [VentureBeat](https://venturebeat.com/ai/flashattention-3-unleashes-the-power-of-h100-gpus-for-llms/).
- TD Cowen analyst predicts Nvidia will outperform expectations due to sustained demand for Hopper-based AI systems, despite concerns about slowing AI hardware spending - [MarketWatch](https://www.marketwatch.com/story/nvidias-stock-gains-as-an-analyst-throws-cold-water-on-this-bearish-concern-2ced7ff2).
- Arrow Electronics' semi-autonomous motorcar project utilizes Nvidia modules for hands-free driving at 200 mph, showcasing advancements in edge AI and autonomous vehicle technology - [EE Times](https://www.eetimes.com/the-ultimate-edge-ai-hands-free-driving-at-200-mph/).

# California AI Safety Bill Sparks Debate and Controversy

- California's AI bill SB-1047 is igniting a fierce debate over AI regulation, with proponents advocating for safety and ethical standards, while opponents fear stifling innovation and consolidation of power among tech giants - [Fortune](https://fortune.com/2024/07/15/california-ai-bill-sb-1047-fierce-debate-regulation-safety/).
- The bill mandates strict guidelines for AI developers, including safety measures, certifications, and incident reporting, enforced by the Attorney General and Labor Commissioner - [The National Law Review](https://natlawreview.com/article/california-dreamin-will-sb-1047-curb-ai-innovation).
- Yann LeCun opposes the bill, warning it could hinder AI research, while figures like Geoffrey Hinton and Yoshua Bengio support it for balancing safety and innovation - [Analytics India Magazine](https://analyticsindiamag.com/ai-news-updates/yann-lecun-urges-signing-letter-to-block-regulation-for-ai-research/).

# US Senate AI Ethics Bill: Key Developments

- The US Senate introduced the "Content Origin Protection and Integrity from Edited and Deepfaked Media Act" (COPIED Act) to regulate AI-generated content and deepfake technology - [TechSpot](https://www.techspot.com/news/103820-us-senate-introduces-ground-breaking-bill-setup-legal.html).
- California's AI bill SB-1047, aimed at regulating powerful AI models, has sparked a fierce debate between safety advocates and innovation proponents - [Fortune](https://fortune.com/2024/07/15/california-ai-bill-sb-1047-fierce-debate-regulation-s

In [136]:
edit_prompt3 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Carefully review each section of the markdown newsletter provided below. 
Each section consists of several bullet points. 

For each section, identify and combine redundant bullet points:

Instructions: 
For each section, identify bullet points containing identical URLs to other bullet points in the same section 
Rewrite the section, combining these similar bullet points to eliminate duplication.
Do not duplicate any URLs within a section.
Check the response carefully and ensure that no links are duplicated within a section.

Newsletter to edit: 
{response_str2}

"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt3
              }],
    n=1,   
    temperature=0.2
)
response_str3 = response.choices[0].message.content
display(Markdown(response_str3.replace("$", "\\$")))


2024-07-16 00:07:16,051 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


```markdown
# AI Stocks Surge Amid Technological Advancements

- Emerging-market equity investors are finding new opportunities in AI beyond US tech giants, with improved governance in markets like Japan and Korea - [AllianceBernstein](https://www.alliancebernstein.com/nz/en/institutions/insights/investment-insights/is-ai-a-new-engine-for-emerging-market-equity-investors.html).
- Despite market volatility, AI remains a dominant investment theme with significant long-term growth potential, impacting productivity gains globally - [Invesco](https://www.invesco.com/hk/en/investment-ideas/quick-takes/ai-series-how-ai-remains-dominant-investment-theme.html).
- Nvidia and Super Micro Computer have seen significant gains, while Alibaba and Innodata are poised for substantial growth in the AI sector - [Yahoo Finance](https://finance.yahoo.com/news/2-artificial-intelligence-ai-stocks-092500064.html).
- Apple's stock rose after being named a "top pick" by Morgan Stanley for its AI efforts, with projections of nearly 500 million iPhone sales over the next two years - [Reddit](https://www.reddit.com/r/technology/comments/1e3u0w3/apple_gains_after_morgan_stanley_calls_stock_top/).
- Vistra Corp. has outperformed with a 1-year total return of over 240%, driven by AI data center power demand and integration of Energy Harbor - [Seeking Alpha](https://seekingalpha.com/article/4704133-vistra-ai-game-changer-is-just-getting-started).
- Arm Holdings and STMicroelectronics are set to benefit from the AI trend, with advancements in AI and 5G technologies - [Barchart](https://www.barchart.com/story/news/27341677/missed-nvidia-2-semiconductor-stocks-that-could-gain-from-the-ai-frenzy).
- Apple stock hits a record high with analysts optimistic about generative AI on iPhones and iPads, projecting up to 500 million iPhone sales in the next two years - [Investopedia](https://www.investopedia.com/apple-stock-price-record-high-analysts-targets-ai-8678016).
- Broadcom is considered a better AI chip buy over Marvell Technology due to its diversified business and stable profits - [The Motley Fool](https://www.fool.com/investing/2024/07/15/better-ai-chip-stock-broadcom-vs-marvell-technolog/).
- Analysts predict a significant AI-driven iPhone upgrade cycle, with potential shipments of nearly 500 million iPhones in fiscal 2025 and 2026 - [CNBC](https://www.cnbc.com/2024/07/15/the-ai-driven-iphone-upgrade-cycle-will-be-huge-two-more-wall-street-firms-agree.html).
- Tech giants like Apple, Microsoft, and Nvidia are driving the stock market surge, with a focus on AI technology - [Forbes](https://www.forbes.com/sites/cio/2024/07/11/as-valuations-around-ai-skyrocket-how-should-cios-look-at-solutions/).
- Lee Se-Dol, a top Go player, was defeated by an AI program in 2016, leading to his retirement in 2019 - [Business Insider](https://www.businessinsider.in/international/news/one-of-the-worlds-greatest-go-players-who-was-defeated-by-ai-warns-that-the-technology-may-not-come-with-a-happy-ending/articleshow/111668042.cms).

# Apple's Stock Price Soars on AI Optimism

- Apple stock hits a record high after receiving price upgrades based on analysts' optimism about generative AI on iPhones and iPads - [Investopedia](https://www.investopedia.com/apple-stock-price-record-high-analysts-targets-ai-8678016).
- Apple's stock rose after being designated as a "top pick" by Morgan Stanley for its AI efforts, with a raised price target boosting device sales - [Reddit](https://www.reddit.com/r/technology/comments/1e3u0w3/apple_gains_after_morgan_stanley_calls_stock_top/).
- Analysts predict a significant iPhone upgrade cycle, with the potential for Apple to ship nearly 500 million iPhones in fiscal 2025 and 2026 combined - [CNBC](https://www.cnbc.com/2024/07/15/the-ai-driven-iphone-upgrade-cycle-will-be-huge-two-more-wall-street-firms-agree.html).
- Tech giants like Apple, Microsoft, and Nvidia are driving the stock market surge, with a focus on AI technology - [Forbes](https://www.forbes.com/sites/cio/2024/07/11/as-valuations-around-ai-skyrocket-how-should-cios-look-at-solutions/).

# Key Developments in AI Policy and Regulation

- The New York Department of Financial Services (NYDFS) mandates that insurance companies' use of AI must not discriminate against consumers - [PYMNTS](https://www.pymnts.com/insurance/2024/nydfs-issues-guidance-for-insurance-companies-use-of-ai/).
- California's AI bill SB-1047, which aims to regulate powerful AI models for safety, is sparking a fierce debate between proponents and opponents - [Fortune](https://fortune.com/2024/07/15/california-ai-bill-sb-1047-fierce-debate-regulation-safety/).
- Yann LeCun opposes California's proposed AI Safety Bill, warning that it could stifle innovation in AI research - [Analytics India Magazine](https://analyticsindiamag.com/ai-news-updates/yann-lecun-urges-signing-letter-to-block-regulation-for-ai-research/).
- OpenAI whistleblowers have filed a complaint with the SEC regarding the company's non-disclosure agreements that limit employees from discussing AI risks - [Boston 25 News](https://www.boston25news.com/news/openai/XENZN4NAQBEHVNVXCDMPCY3RXQ/).
- The US Senate introduced the "Content Origin Protection and Integrity from Edited and Deepfaked Media Act" (COPIED Act) to regulate the use of AI-generated content and deepfake technology - [TechSpot](https://www.techspot.com/news/103820-us-senate-introduces-ground-breaking-bill-setup-legal.html).

# Google AI Overviews: Current Trends and Developments

- Google has significantly reduced the display of AI Overviews in Search, now appearing in less than 10% of search queries - [9to5Google](https://9to5google.com/2024/07/15/google-ai-overviews-decline-report-june-2024/).
- Google AI Overviews now appear less than 7% of the time, with significant drops in visibility on education, entertainment, and ecommerce queries - [Search Engine Land](https://searchengineland.com/google-ai-overviews-visibility-new-low-444048).
- Google has introduced AI Overviews, a new version of its search engine, causing concerns among businesses about its impact on web traffic, advertising revenue, and visibility for smaller businesses online - [Tech in Asia](https://www.techinasia.com/error-404-web-killed-googles-ai-search).
- Google has launched Vids, an AI-powered video maker, in Workspace Labs, allowing users to create presentation videos by combining docs, slides, voiceovers, and video recordings - [The Verge](https://www.theverge.com/2024/7/15/24199063/google-vids-gemini-ai-app-workspace-labs-available).

# AI in Healthcare: Today's Innovations and Challenges

- Kaiser Permanente uses advanced AI for early intervention in patient care, enhancing workflow and saving lives - [VentureBeat](https://venturebeat.com/ai/how-kaiser-permanente-is-using-gen-ai-to-paradoxically-make-care-more-human-again/)
- Cambridge scientists' AI tool predicts Alzheimer's progression with higher accuracy than current clinical markers - [Medical Xpress](https://medicalxpress.com/news/2024-07-artificial-intelligence-outperforms-clinical-alzheimer.html)
- Thrive AI Health's personalized AI health coach aims to address chronic conditions through behavior change - [Fortune](https://fortune.com/well/2024/07/08/thrive-ai-health-coach-sam-altman-arianna-huffington/)
- NYDFS mandates non-discriminatory AI use in insurance, emphasizing transparency and fairness - [PYMNTS](https://www.pymnts.com/insurance/2024/nydfs-issues-guidance-for-insurance-companies-use-of-ai/)
- Fast Forward accelerator supports AI startups tackling global issues, focusing on ethical AI deployment - [Triple Pundit](https://www.triplepundit.com/story/2024/artificial-intelligence-Fast-Forward/805751)

# Samsung AI Innovations: A New Era in Tech

- Samsung introduces Galaxy Ring, a smart ring with AI capabilities for health and fitness monitoring, featuring a lightweight titanium frame and up to seven days of battery life - [Women's Wear Daily](https://wwd.com/business-news/technology/samsungs-galaxy-ring-ai-wearable-jewelry-1236492272/)
- Samsung confirms older Galaxy devices, including the S22, S23, and S24 series, will receive the One UI 6.1.1 update with new AI features like Generative AI for camera enhancements - [Android Central](https://www.androidcentral.com/apps-software/samsung-one-ui-6-1-1-devices-confirmed)
- One UI 6.1.1 introduces AI suggested replies for quick responses on Galaxy Z Flip 6 and One UI Watch 6, ensuring privacy by processing messages on-device - [Android Authority](https://www.androidauthority.com/samsung-suggested-replies-one-ui-3460745/)
- Samsung leaves Galaxy S21 out of the One UI 6.1.1 update, focusing AI enhancements on newer models like the S22 and Z Fold series due to potential processing power limitations - [Phone Arena](https://www.phonearena.com/news/Samsung-leaves-Galaxy-S21-out-of-the-One-UI-6.1.1-update-AI-party_id160416)
- The Samsung Galaxy Ring has sold out on Samsung's online store, with limited supplies at third-party retailers like Best Buy, due to its positive reviews and lack of a subscription model - [TechRadar](https://www.techradar.com/health-fitness/its-not-you-galaxy-ring-just-sold-out-from-samsung-but-weve-found-another-way-for-you-to-put-a-ring-on-it)

# The Future of Work and AI: Transformative Trends

- AI programs like ChatGPT and Microsoft Copilot are enhancing productivity by automating tasks and improving workflows - [SoGlos](https://www.soglos.com/interview/business/ai-is-going-to-become-a-game-changer-in-the-workplace-says-leading-cheltenham-tech-firm/23158/).
- Businesses are evolving towards neural organizations, emphasizing interconnectedness for innovation and speed - [McKinsey & Company](https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/gen-ai-and-beyond-where-else-to-focus-now).
- 77% of professionals believe AI will significantly impact their work in the next five years, though trust in AI for autonomous decision-making remains low - [TechRepublic](https://www.techrepublic.com/article/thomson-reuters-ai-report/).
- The food industry is investing heavily in AI and automation to improve service and reduce costs, with companies like McDonald’s leading the way - [CNBC](https://www.cnbc.com/2024/07/15/how-ai-and-automation-will-reshape-grocery-stores-and-fast-food-chains.html).
- AI advancements in finance are seen as a potential solution to banking discrimination and job displacement concerns - [Newsweek](https://www.newsweek.com/sam-altman-ai-ethics-minimize-discrimination-banking-create-jobs-1923321).
- Tech giants like Apple, Microsoft, and Nvidia are driving a stock market surge with their focus on AI technology - [Forbes](https://www.forbes.com/sites/cio/2024/07/11/as-valuations-around-ai-skyrocket-how-should-cios-look-at-solutions/).
- Fast Forward's AI for Humanity initiative supports startups using AI to address global issues such as poverty and healthcare inequity - [Triple Pundit](https://www.triplepundit.com/story/2024/artificial-intelligence-Fast-Forward/805751).
- Instacart's AI-powered smart carts are being introduced to more grocery stores, enhancing shopping experiences with real-time recommendations and gamification - [Fortune](https://fortune.com/2024/07/08/instacart-ai-powered-caper-carts-artificial-intelligence-gamification-grocery-stores/).

# AI Chipmakers: Market Trends and Innovations

- Companies are heavily investing in AI accelerators, with forecasts predicting the datacenter AI accelerator market to reach billions by 2027 - [The Next Platform](https://www.nextplatform.com/2024/07/15/ongoing-saga-how-much-money-will-be-spent-on-ai-chips/).
- Arm Holdings and STMicroelectronics are poised to benefit from the AI trend, with ARM showing remarkable revenue growth - [Barchart](https://www.barchart.com/story/news/27341677/missed-nvidia-2-semiconductor-stocks-that-could-gain-from-the-ai-frenzy).
- Broadcom is considered a better AI chip buy than Marvell Technology due to its diversified business and stable profits - [The Motley Fool](https://www.fool.com/investing/2024/07/15/better-ai-chip-stock-broadcom-vs-marvell-technolog/).
- Nvidia's stock gains as analysts remain optimistic about its earnings, driven by strong demand for data center AI systems - [MarketWatch](https://www.marketwatch.com/story/nvidias-stock-gains-as-an-analyst-throws-cold-water-on-this-bearish-concern-2ced7ff2).
- AMD has launched the Ryzen AI 300 series, featuring new Zen 5 cores and RDNA 3.5 GPU, enhancing AI capabilities - [Serve The Home](https://www.servethehome.com/amd-ryzen-ai-300-series-launched/).

# OpenAI Ethics Investigation: Key Developments

- OpenAI whistleblowers are urging the SEC to investigate the company for alleged violations related to prohibiting whistleblowing about safety risks - [Mashable](https://sea.mashable.com/tech/33422/openai-whistleblowers-call-on-sec-to-investigate-the-ai-company).
- U.S. Sen. Chuck Grassley highlighted concerns about OpenAI's policies silencing whistleblowers and emphasized the need for change in their NDA practices - [Boston 25 News](https://www.boston25news.com/news/openai/XENZN4NAQBEHVNVXCDMPCY3RXQ/).
- OpenAI is reportedly progressing rapidly towards human-level reasoning, unveiling a system to track progress towards A.G.I., with plans for models to act as agents, innovators, and organizations - [Observer](https://observer.com/2024/07/openai-employees-concerns-strawberry-ai-project/).
- An AI ethics council, co-chaired by Sam Altman of OpenAI and John Hope Bryant, aims to ensure underserved communities are not left behind by AI - [AJC](https://www.ajc.com/news/business/ai-ethics-council-hosts-inaugural-meeting-in-atlanta-looks-to-expand/QCAFZQUALJAJHJUJFZUVSO3XSA/).
- The US Senate introduced the "Content Origin Protection and Integrity from Edited and Deepfaked Media Act" (COPIED Act) to address concerns over unethical use of AI-generated content and deepfake technology - [TechSpot](https://www.techspot.com/news/103820-us-senate-introduces-ground-breaking-bill-setup-legal.html).

# Nvidia AI Gold Rush: Key Developments

- Providers of AI infrastructure are benefiting from the growing demand in the Nvidia-led AI gold rush, with hyperscale data centers projected to grow to \$262 billion by 2032 - [MarketWatch](https://www.marketwatch.com/story/the-nvidia-led-ai-gold-rush-is-paydirt-for-these-5-data-center-growth-stocks-8f30534c).
- Wall Street analysts remain bullish on Nvidia stock, despite investor caution, with expectations of strong fiscal Q2 results driven by AI system demand - [Investor's Business Daily](https://www.investors.com/news/technology/nvidia-stock-nvda-analysts-bullish/).
- FlashAttention-3 optimizes attention computation for large language models using Nvidia Hopper GPUs, achieving significant speedups and efficiency improvements - [VentureBeat](https://venturebeat.com/ai/flashattention-3-unleashes-the-power-of-h100-gpus-for-llms/).
- TD Cowen analyst predicts Nvidia will outperform expectations due to sustained demand for Hopper-based AI systems, despite concerns about slowing AI hardware spending - [MarketWatch](https://www.marketwatch.com/story/nvidias-stock-gains-as-an-analyst-throws-cold-water-on-this-bearish-concern-2ced7ff2).
- Arrow Electronics' semi-autonomous motorcar project utilizes Nvidia modules for hands-free driving at 200 mph, showcasing advancements in edge AI and autonomous vehicle technology - [EE Times](https://www.eetimes.com/the-ultimate-edge-ai-hands-free-driving-at-200-mph/).

# California AI Safety Bill Sparks Debate and Controversy

- California's AI bill SB-1047 is igniting a fierce debate over AI regulation, with proponents advocating for safety and ethical standards, while opponents fear stifling innovation and consolidation of power among tech giants - [Fortune](https://fortune.com/2024/07/15/california-ai-bill-sb-1047-fierce-debate-regulation-safety/).
- The bill mandates strict guidelines for AI developers, including safety measures, certifications, and incident reporting, enforced by the Attorney General and Labor Commissioner - [The National Law Review](https://natlawreview.com/article/california-dreamin-will-sb-1047-curb-ai-innovation).
- Yann LeCun opposes the bill, warning it could hinder AI research, while figures like Geoffrey Hinton and Yoshua Bengio support it for balancing safety and innovation - [Analytics India Magazine](https://analyticsindiamag.com/ai-news-updates/yann-lecun-urges-signing-letter-to-block-regulation-for-ai-research/).

# US Senate AI Ethics Bill: Key Developments

- The US Senate introduced the "Content Origin Protection and Integrity from Edited and Deepfaked Media Act" (COPIED Act) to regulate AI-generated content and deepfake technology - [TechSpot](https://www.techspot.com/news/103820-us-senate-introduces-ground-breaking-bill-setup-legal.html).
- California's AI bill SB-1047, aimed at regulating powerful AI models, has sparked a fierce debate between safety advocates and innovation proponents - [Fortune](https://fortune.com/2024/07/15/california-ai-bill-sb-1047-fierce-debate-regulation-s

In [137]:
display(Markdown(response_str3[11:].replace("$", "\\$")))



# AI Stocks Surge Amid Technological Advancements

- Emerging-market equity investors are finding new opportunities in AI beyond US tech giants, with improved governance in markets like Japan and Korea - [AllianceBernstein](https://www.alliancebernstein.com/nz/en/institutions/insights/investment-insights/is-ai-a-new-engine-for-emerging-market-equity-investors.html).
- Despite market volatility, AI remains a dominant investment theme with significant long-term growth potential, impacting productivity gains globally - [Invesco](https://www.invesco.com/hk/en/investment-ideas/quick-takes/ai-series-how-ai-remains-dominant-investment-theme.html).
- Nvidia and Super Micro Computer have seen significant gains, while Alibaba and Innodata are poised for substantial growth in the AI sector - [Yahoo Finance](https://finance.yahoo.com/news/2-artificial-intelligence-ai-stocks-092500064.html).
- Apple's stock rose after being named a "top pick" by Morgan Stanley for its AI efforts, with projections of nearly 500 million iPhone sales over the next two years - [Reddit](https://www.reddit.com/r/technology/comments/1e3u0w3/apple_gains_after_morgan_stanley_calls_stock_top/).
- Vistra Corp. has outperformed with a 1-year total return of over 240%, driven by AI data center power demand and integration of Energy Harbor - [Seeking Alpha](https://seekingalpha.com/article/4704133-vistra-ai-game-changer-is-just-getting-started).
- Arm Holdings and STMicroelectronics are set to benefit from the AI trend, with advancements in AI and 5G technologies - [Barchart](https://www.barchart.com/story/news/27341677/missed-nvidia-2-semiconductor-stocks-that-could-gain-from-the-ai-frenzy).
- Apple stock hits a record high with analysts optimistic about generative AI on iPhones and iPads, projecting up to 500 million iPhone sales in the next two years - [Investopedia](https://www.investopedia.com/apple-stock-price-record-high-analysts-targets-ai-8678016).
- Broadcom is considered a better AI chip buy over Marvell Technology due to its diversified business and stable profits - [The Motley Fool](https://www.fool.com/investing/2024/07/15/better-ai-chip-stock-broadcom-vs-marvell-technolog/).
- Analysts predict a significant AI-driven iPhone upgrade cycle, with potential shipments of nearly 500 million iPhones in fiscal 2025 and 2026 - [CNBC](https://www.cnbc.com/2024/07/15/the-ai-driven-iphone-upgrade-cycle-will-be-huge-two-more-wall-street-firms-agree.html).
- Tech giants like Apple, Microsoft, and Nvidia are driving the stock market surge, with a focus on AI technology - [Forbes](https://www.forbes.com/sites/cio/2024/07/11/as-valuations-around-ai-skyrocket-how-should-cios-look-at-solutions/).
- Lee Se-Dol, a top Go player, was defeated by an AI program in 2016, leading to his retirement in 2019 - [Business Insider](https://www.businessinsider.in/international/news/one-of-the-worlds-greatest-go-players-who-was-defeated-by-ai-warns-that-the-technology-may-not-come-with-a-happy-ending/articleshow/111668042.cms).

# Apple's Stock Price Soars on AI Optimism

- Apple stock hits a record high after receiving price upgrades based on analysts' optimism about generative AI on iPhones and iPads - [Investopedia](https://www.investopedia.com/apple-stock-price-record-high-analysts-targets-ai-8678016).
- Apple's stock rose after being designated as a "top pick" by Morgan Stanley for its AI efforts, with a raised price target boosting device sales - [Reddit](https://www.reddit.com/r/technology/comments/1e3u0w3/apple_gains_after_morgan_stanley_calls_stock_top/).
- Analysts predict a significant iPhone upgrade cycle, with the potential for Apple to ship nearly 500 million iPhones in fiscal 2025 and 2026 combined - [CNBC](https://www.cnbc.com/2024/07/15/the-ai-driven-iphone-upgrade-cycle-will-be-huge-two-more-wall-street-firms-agree.html).
- Tech giants like Apple, Microsoft, and Nvidia are driving the stock market surge, with a focus on AI technology - [Forbes](https://www.forbes.com/sites/cio/2024/07/11/as-valuations-around-ai-skyrocket-how-should-cios-look-at-solutions/).

# Key Developments in AI Policy and Regulation

- The New York Department of Financial Services (NYDFS) mandates that insurance companies' use of AI must not discriminate against consumers - [PYMNTS](https://www.pymnts.com/insurance/2024/nydfs-issues-guidance-for-insurance-companies-use-of-ai/).
- California's AI bill SB-1047, which aims to regulate powerful AI models for safety, is sparking a fierce debate between proponents and opponents - [Fortune](https://fortune.com/2024/07/15/california-ai-bill-sb-1047-fierce-debate-regulation-safety/).
- Yann LeCun opposes California's proposed AI Safety Bill, warning that it could stifle innovation in AI research - [Analytics India Magazine](https://analyticsindiamag.com/ai-news-updates/yann-lecun-urges-signing-letter-to-block-regulation-for-ai-research/).
- OpenAI whistleblowers have filed a complaint with the SEC regarding the company's non-disclosure agreements that limit employees from discussing AI risks - [Boston 25 News](https://www.boston25news.com/news/openai/XENZN4NAQBEHVNVXCDMPCY3RXQ/).
- The US Senate introduced the "Content Origin Protection and Integrity from Edited and Deepfaked Media Act" (COPIED Act) to regulate the use of AI-generated content and deepfake technology - [TechSpot](https://www.techspot.com/news/103820-us-senate-introduces-ground-breaking-bill-setup-legal.html).

# Google AI Overviews: Current Trends and Developments

- Google has significantly reduced the display of AI Overviews in Search, now appearing in less than 10% of search queries - [9to5Google](https://9to5google.com/2024/07/15/google-ai-overviews-decline-report-june-2024/).
- Google AI Overviews now appear less than 7% of the time, with significant drops in visibility on education, entertainment, and ecommerce queries - [Search Engine Land](https://searchengineland.com/google-ai-overviews-visibility-new-low-444048).
- Google has introduced AI Overviews, a new version of its search engine, causing concerns among businesses about its impact on web traffic, advertising revenue, and visibility for smaller businesses online - [Tech in Asia](https://www.techinasia.com/error-404-web-killed-googles-ai-search).
- Google has launched Vids, an AI-powered video maker, in Workspace Labs, allowing users to create presentation videos by combining docs, slides, voiceovers, and video recordings - [The Verge](https://www.theverge.com/2024/7/15/24199063/google-vids-gemini-ai-app-workspace-labs-available).

# AI in Healthcare: Today's Innovations and Challenges

- Kaiser Permanente uses advanced AI for early intervention in patient care, enhancing workflow and saving lives - [VentureBeat](https://venturebeat.com/ai/how-kaiser-permanente-is-using-gen-ai-to-paradoxically-make-care-more-human-again/)
- Cambridge scientists' AI tool predicts Alzheimer's progression with higher accuracy than current clinical markers - [Medical Xpress](https://medicalxpress.com/news/2024-07-artificial-intelligence-outperforms-clinical-alzheimer.html)
- Thrive AI Health's personalized AI health coach aims to address chronic conditions through behavior change - [Fortune](https://fortune.com/well/2024/07/08/thrive-ai-health-coach-sam-altman-arianna-huffington/)
- NYDFS mandates non-discriminatory AI use in insurance, emphasizing transparency and fairness - [PYMNTS](https://www.pymnts.com/insurance/2024/nydfs-issues-guidance-for-insurance-companies-use-of-ai/)
- Fast Forward accelerator supports AI startups tackling global issues, focusing on ethical AI deployment - [Triple Pundit](https://www.triplepundit.com/story/2024/artificial-intelligence-Fast-Forward/805751)

# Samsung AI Innovations: A New Era in Tech

- Samsung introduces Galaxy Ring, a smart ring with AI capabilities for health and fitness monitoring, featuring a lightweight titanium frame and up to seven days of battery life - [Women's Wear Daily](https://wwd.com/business-news/technology/samsungs-galaxy-ring-ai-wearable-jewelry-1236492272/)
- Samsung confirms older Galaxy devices, including the S22, S23, and S24 series, will receive the One UI 6.1.1 update with new AI features like Generative AI for camera enhancements - [Android Central](https://www.androidcentral.com/apps-software/samsung-one-ui-6-1-1-devices-confirmed)
- One UI 6.1.1 introduces AI suggested replies for quick responses on Galaxy Z Flip 6 and One UI Watch 6, ensuring privacy by processing messages on-device - [Android Authority](https://www.androidauthority.com/samsung-suggested-replies-one-ui-3460745/)
- Samsung leaves Galaxy S21 out of the One UI 6.1.1 update, focusing AI enhancements on newer models like the S22 and Z Fold series due to potential processing power limitations - [Phone Arena](https://www.phonearena.com/news/Samsung-leaves-Galaxy-S21-out-of-the-One-UI-6.1.1-update-AI-party_id160416)
- The Samsung Galaxy Ring has sold out on Samsung's online store, with limited supplies at third-party retailers like Best Buy, due to its positive reviews and lack of a subscription model - [TechRadar](https://www.techradar.com/health-fitness/its-not-you-galaxy-ring-just-sold-out-from-samsung-but-weve-found-another-way-for-you-to-put-a-ring-on-it)

# The Future of Work and AI: Transformative Trends

- AI programs like ChatGPT and Microsoft Copilot are enhancing productivity by automating tasks and improving workflows - [SoGlos](https://www.soglos.com/interview/business/ai-is-going-to-become-a-game-changer-in-the-workplace-says-leading-cheltenham-tech-firm/23158/).
- Businesses are evolving towards neural organizations, emphasizing interconnectedness for innovation and speed - [McKinsey & Company](https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/gen-ai-and-beyond-where-else-to-focus-now).
- 77% of professionals believe AI will significantly impact their work in the next five years, though trust in AI for autonomous decision-making remains low - [TechRepublic](https://www.techrepublic.com/article/thomson-reuters-ai-report/).
- The food industry is investing heavily in AI and automation to improve service and reduce costs, with companies like McDonald’s leading the way - [CNBC](https://www.cnbc.com/2024/07/15/how-ai-and-automation-will-reshape-grocery-stores-and-fast-food-chains.html).
- AI advancements in finance are seen as a potential solution to banking discrimination and job displacement concerns - [Newsweek](https://www.newsweek.com/sam-altman-ai-ethics-minimize-discrimination-banking-create-jobs-1923321).
- Tech giants like Apple, Microsoft, and Nvidia are driving a stock market surge with their focus on AI technology - [Forbes](https://www.forbes.com/sites/cio/2024/07/11/as-valuations-around-ai-skyrocket-how-should-cios-look-at-solutions/).
- Fast Forward's AI for Humanity initiative supports startups using AI to address global issues such as poverty and healthcare inequity - [Triple Pundit](https://www.triplepundit.com/story/2024/artificial-intelligence-Fast-Forward/805751).
- Instacart's AI-powered smart carts are being introduced to more grocery stores, enhancing shopping experiences with real-time recommendations and gamification - [Fortune](https://fortune.com/2024/07/08/instacart-ai-powered-caper-carts-artificial-intelligence-gamification-grocery-stores/).

# AI Chipmakers: Market Trends and Innovations

- Companies are heavily investing in AI accelerators, with forecasts predicting the datacenter AI accelerator market to reach billions by 2027 - [The Next Platform](https://www.nextplatform.com/2024/07/15/ongoing-saga-how-much-money-will-be-spent-on-ai-chips/).
- Arm Holdings and STMicroelectronics are poised to benefit from the AI trend, with ARM showing remarkable revenue growth - [Barchart](https://www.barchart.com/story/news/27341677/missed-nvidia-2-semiconductor-stocks-that-could-gain-from-the-ai-frenzy).
- Broadcom is considered a better AI chip buy than Marvell Technology due to its diversified business and stable profits - [The Motley Fool](https://www.fool.com/investing/2024/07/15/better-ai-chip-stock-broadcom-vs-marvell-technolog/).
- Nvidia's stock gains as analysts remain optimistic about its earnings, driven by strong demand for data center AI systems - [MarketWatch](https://www.marketwatch.com/story/nvidias-stock-gains-as-an-analyst-throws-cold-water-on-this-bearish-concern-2ced7ff2).
- AMD has launched the Ryzen AI 300 series, featuring new Zen 5 cores and RDNA 3.5 GPU, enhancing AI capabilities - [Serve The Home](https://www.servethehome.com/amd-ryzen-ai-300-series-launched/).

# OpenAI Ethics Investigation: Key Developments

- OpenAI whistleblowers are urging the SEC to investigate the company for alleged violations related to prohibiting whistleblowing about safety risks - [Mashable](https://sea.mashable.com/tech/33422/openai-whistleblowers-call-on-sec-to-investigate-the-ai-company).
- U.S. Sen. Chuck Grassley highlighted concerns about OpenAI's policies silencing whistleblowers and emphasized the need for change in their NDA practices - [Boston 25 News](https://www.boston25news.com/news/openai/XENZN4NAQBEHVNVXCDMPCY3RXQ/).
- OpenAI is reportedly progressing rapidly towards human-level reasoning, unveiling a system to track progress towards A.G.I., with plans for models to act as agents, innovators, and organizations - [Observer](https://observer.com/2024/07/openai-employees-concerns-strawberry-ai-project/).
- An AI ethics council, co-chaired by Sam Altman of OpenAI and John Hope Bryant, aims to ensure underserved communities are not left behind by AI - [AJC](https://www.ajc.com/news/business/ai-ethics-council-hosts-inaugural-meeting-in-atlanta-looks-to-expand/QCAFZQUALJAJHJUJFZUVSO3XSA/).
- The US Senate introduced the "Content Origin Protection and Integrity from Edited and Deepfaked Media Act" (COPIED Act) to address concerns over unethical use of AI-generated content and deepfake technology - [TechSpot](https://www.techspot.com/news/103820-us-senate-introduces-ground-breaking-bill-setup-legal.html).

# Nvidia AI Gold Rush: Key Developments

- Providers of AI infrastructure are benefiting from the growing demand in the Nvidia-led AI gold rush, with hyperscale data centers projected to grow to \$262 billion by 2032 - [MarketWatch](https://www.marketwatch.com/story/the-nvidia-led-ai-gold-rush-is-paydirt-for-these-5-data-center-growth-stocks-8f30534c).
- Wall Street analysts remain bullish on Nvidia stock, despite investor caution, with expectations of strong fiscal Q2 results driven by AI system demand - [Investor's Business Daily](https://www.investors.com/news/technology/nvidia-stock-nvda-analysts-bullish/).
- FlashAttention-3 optimizes attention computation for large language models using Nvidia Hopper GPUs, achieving significant speedups and efficiency improvements - [VentureBeat](https://venturebeat.com/ai/flashattention-3-unleashes-the-power-of-h100-gpus-for-llms/).
- TD Cowen analyst predicts Nvidia will outperform expectations due to sustained demand for Hopper-based AI systems, despite concerns about slowing AI hardware spending - [MarketWatch](https://www.marketwatch.com/story/nvidias-stock-gains-as-an-analyst-throws-cold-water-on-this-bearish-concern-2ced7ff2).
- Arrow Electronics' semi-autonomous motorcar project utilizes Nvidia modules for hands-free driving at 200 mph, showcasing advancements in edge AI and autonomous vehicle technology - [EE Times](https://www.eetimes.com/the-ultimate-edge-ai-hands-free-driving-at-200-mph/).

# California AI Safety Bill Sparks Debate and Controversy

- California's AI bill SB-1047 is igniting a fierce debate over AI regulation, with proponents advocating for safety and ethical standards, while opponents fear stifling innovation and consolidation of power among tech giants - [Fortune](https://fortune.com/2024/07/15/california-ai-bill-sb-1047-fierce-debate-regulation-safety/).
- The bill mandates strict guidelines for AI developers, including safety measures, certifications, and incident reporting, enforced by the Attorney General and Labor Commissioner - [The National Law Review](https://natlawreview.com/article/california-dreamin-will-sb-1047-curb-ai-innovation).
- Yann LeCun opposes the bill, warning it could hinder AI research, while figures like Geoffrey Hinton and Yoshua Bengio support it for balancing safety and innovation - [Analytics India Magazine](https://analyticsindiamag.com/ai-news-updates/yann-lecun-urges-signing-letter-to-block-regulation-for-ai-research/).

# US Senate AI Ethics Bill: Key Developments

- The US Senate introduced the "Content Origin Protection and Integrity from Edited and Deepfaked Media Act" (COPIED Act) to regulate AI-generated content and deepfake technology - [TechSpot](https://www.techspot.com/news/103820-us-senate-introduces-ground-breaking-bill-setup-legal.html).
- California's AI bill SB-1047, aimed at regulating powerful AI models, has sparked a fierce debate between safety advocates and innovation proponents - [Fortune](https://fortune.com/2024/07/15/california-ai-bill-sb-1047-fierce-debate-regulation-s

In [138]:
response_str3.replace("$", "\\$")

'```markdown\n# AI Stocks Surge Amid Technological Advancements\n\n- Emerging-market equity investors are finding new opportunities in AI beyond US tech giants, with improved governance in markets like Japan and Korea - [AllianceBernstein](https://www.alliancebernstein.com/nz/en/institutions/insights/investment-insights/is-ai-a-new-engine-for-emerging-market-equity-investors.html).\n- Despite market volatility, AI remains a dominant investment theme with significant long-term growth potential, impacting productivity gains globally - [Invesco](https://www.invesco.com/hk/en/investment-ideas/quick-takes/ai-series-how-ai-remains-dominant-investment-theme.html).\n- Nvidia and Super Micro Computer have seen significant gains, while Alibaba and Innodata are poised for substantial growth in the AI sector - [Yahoo Finance](https://finance.yahoo.com/news/2-artificial-intelligence-ai-stocks-092500064.html).\n- Apple\'s stock rose after being named a "top pick" by Morgan Stanley for its AI efforts